# JUICE RPWI HF SID3 (Full): L1a QL -- 2025/3/21

# Import lib

In [ ]:
import copy
import datetime
import math
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import sys

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf
import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cal_lib   as juice_cal
import juice_cdf_lib   as juice_cdf
import juice_math_lib  as juice_math
import juice_sid3_data as juice_data
import juice_sid3_lib  as juice_sid3
import juice_spec_lib  as juice_spec

# Setting and Read CDF file: set by User

In [ ]:
# *** Dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump
# *** CAL ***
unit_mode = 0                           # [Power]     0: raw     1: V＠ADC     2: V@HF    3: V@RWI  4: V/m
band_mode = 0                           # [Power]     0: sum     1: /Hz
cal_mode  = 2                           # [Power]     0: background     1: CAL     2: all
# *** Unit ***
f_mode    = 1                           # [Frequency] 0: linear  1: log
time_mode = 0                           # [Time]      0: Epoch   1: data number
gap_mode  = 1                           # [gap]       0: n/a     1: gap
# *** Frenquency in Linear ***
f_mode_min = 80;  f_mode_max = 2068     # 80 - 2068 : as same as SID-4/20
# *** Power range set ***
p_raw_max = 9.5                         # background: 7.5   CAL: 10
p_raw_min = 2.0                         # background: 2.5   CAL: 5
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
N_ch0     = 4                           # [channel]   2: 2-ch    3: 3-ch                              >3: any
comp_mode = 4                           # [Complex]   0: Poweer  1: Matrix   2: PolSep  3: Matrix-2D  >3: any

In [ ]:
E_min = '2024-08-20 00:00:00';  t_min0 = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S")
E_max = '2024-08-24 00:00:00';  t_max0 = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S")
t_min0 = 0;  t_max0 = 0

# get CDF data

In [ ]:
# date='20230420';  ver = 'V03'
# date='20230523';  ver = 'V03'
# date='20230525';  ver = 'V03'
# date='20240126';  ver = 'V02'
# date='20240701';  ver = 'V02'
# date='20240706';  ver = 'V02'
# date='20240819';  ver = 'V02'
# date='20240820';  ver = 'V02'
# date='20240822';  ver = 'V02'   # CAL
# date='20240823';  ver = 'V02'
date='20240909';  ver = 'V02'
# date='0';         ver = 'V02'   # Test data
data_dir, data_name_list = juice_data.datalist(date, ver)       # [date]   yyyymmdd: group read    others: file list

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)
for i in range(num_list):
    data_name = data_name_list[i];  cdf_file = data_dir + data_name;  print(i, cdf_file)
    RPWI_FSW_version = juice_cdf._RPWI_FSW_version(cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid3.hf_sid3_read(cdf, RPWI_FSW_version)
    if i==0:
        data = data1;                                print(data.EuEu.shape)
    else:
        data = juice_sid3.hf_sid3_add(data, data1);  print(data.EuEu.shape)
data_name = os.path.split(data_name)[1];             print("data name:", data_name)

In [ ]:
data = juice_sid3.hf_sid3_shaping(data, cal_mode, N_ch0, comp_mode)

In [ ]:
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
n_time0 = data.EuEu.shape[0];    n_freq0 = data.EuEu.shape[1]
f_min0  = data.frequency[0][0];  f_max0  = (max(np.ravel(data.frequency)))

print("       date and time:", str_date)
print("           data size:", data.EuEu.shape, "  [ Num-samples:", n_time0, "  Num-Frequency:", n_freq0, "]")
print("           Frequency:", f_min0,  "-", f_max0, "kHz")
print("f, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0], data.freq_step[0][0])
print("                      [BAND0] ", 'start {:4d} '.format(data.B0_startf[0]), 'stop {:5d} '.format(data.B0_stopf[0]), 'step {:3d} '.format(data.B0_step[0]), 'repeat {:2d} '.format(data.B0_repeat[0]),  'subdiv {:2d}'.format(data.B0_subdiv[0]), ) 
print("                      [BAND1] ", 'start {:4d} '.format(data.B1_startf[0]), 'stop {:5d} '.format(data.B1_stopf[0]), 'step {:3d} '.format(data.B1_step[0]), 'repeat {:2d} '.format(data.B1_repeat[0]),  'subdiv {:2d}'.format(data.B1_subdiv[0]), ) 
print("                      [BAND2] ", 'start {:4d} '.format(data.B2_startf[0]), 'stop {:5d} '.format(data.B2_stopf[0]), 'step {:3d} '.format(data.B2_step[0]), 'repeat {:2d} '.format(data.B2_repeat[0]),  'subdiv {:2d}'.format(data.B2_subdiv[0]), ) 
print("                      [BAND3] ", 'start {:4d} '.format(data.B3_startf[0]), 'stop {:5d} '.format(data.B3_stopf[0]), 'step {:3d} '.format(data.B3_step[0]), 'repeat {:2d} '.format(data.B3_repeat[0]),  'subdiv {:2d}'.format(data.B3_subdiv[0]), ) 
print("                      [BAND4] ", 'start {:4d} '.format(data.B4_startf[0]), 'stop {:5d} '.format(data.B4_stopf[0]), 'step {:3d} '.format(data.B4_step[0]), 'repeat {:2d} '.format(data.B4_repeat[0]),  'subdiv {:2d}'.format(data.B4_subdiv[0]), ) 

if N_ch0     > 3:   N_ch0     = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
if comp_mode > 3:   comp_mode = data.complex[0]
print("                  Ch:", N_ch0, "  [U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], "]")
#
N_frequency = data.frequency.shape[1]
if (comp_mode == 0):
    if N_ch0 == 3:
        print("             complex:", comp_mode, " (power 3-ch) <Table 6.2.2>")
    else:
        print("             complex:", comp_mode, " (power 2-ch) <Table 6.2.3>")
    N_component = N_ch0;          print("       ==> Component:", N_component)
elif (comp_mode == 1):
    if N_ch0 == 3:
        print("             complex:", comp_mode, " (nominal spectral matrix 3-ch) <Table 6.2.4>")
    else:
        print("             complex:", comp_mode, " (nominal spectral matrix 2-ch) <Table 6.2.5>")
    N_component = N_ch0 * N_ch0;  print("       ==> Component:", N_component)
elif (comp_mode == 2):
    if N_ch0 == 3:
        print("             complex:", comp_mode, " (Polarization separation 3-ch) <Table 6.2.6>")
        N_component = 27 + 3;     print("       ==> Component:", N_component)
    else:
        print("             complex:", comp_mode, " (Polarization separation 2-ch) <Table 6.2.7>")
        N_component = 12 + 3;     print("       ==> Component:", N_component)
else:
    print(    "             complex:", comp_mode, " (3D spectral matrix 3-ch) <Table 6.2.8>")
    N_component = 21;             print("       ==> Component:", N_component)
#
print("         BG_downlink:", data.BG_downlink[0])
print("       RFI_rejection:", data.RFI_rejection[0])

# Spectrum Data

In [ ]:
T_HF  = data.T_HF_FPGA[n_time0//2];  T_RWI = (data.T_RWI_CH1[n_time0//2] + data.T_RWI_CH2[n_time0//2])/2
print("Temperature(org): ", 'HF - {:.1f}'.format(T_HF), '  RWI - {:.1f}'.format(T_RWI))
if T_HF  > 199 or T_HF  < -50 or math.isnan(T_HF):
   T_HF  = 25;  data.T_HF_FPGA[:] = math.nan
if T_RWI > 199 or T_RWI < -199 or math.isnan(T_RWI):
   T_RWI = 25;  data.T_RWI_CH1[:] = math.nan;  data.T_RWI_CH2[:] = math.nan
print("Temperature(cal): ", 'HF - {:.1f}'.format(T_HF), '  RWI - {:.1f}'.format(T_RWI))

In [ ]:
spec = copy.deepcopy(data)
spec = juice_cal.spec_cal(spec,   3, unit_mode, band_mode, T_HF, T_RWI)     # CAL
spec.EE = spec.EuEu + spec.EvEv + spec.EwEw

Epoch_1d  = spec.epoch.tolist();      n_time1 = spec.EuEu.shape[0];  num_1d = np.arange(n_time1)
freq_1d   = spec.freq  [n_time1//2];  n_freq1 = freq_1d.shape[0];    f_min0 = freq_1d[0];  f_max0  = freq_1d[-1]
freq_w_1d = spec.freq_w[n_time1//2]

# NAN
if gap_mode == 1 and time_mode == 0:
    for i in range(n_time1-1):
        if spec.scet[i+1]-spec.scet[i] > 60:
            print("[gap]", i, spec.epoch[i], i+1, spec.epoch[i+1])
            juice_sid3.spec_nan(spec, i);   juice_sid3.spec_nan(spec, i+1)

In [ ]:
spec = juice_spec.hf_getspec_angle    (spec, 3)     # Coherency & ange
spec = juice_spec.hf_getspec_stokes   (spec, 3)     # Stokes and Polarization Parameters 
spec = juice_spec.hf_getspec_stokes_3d(spec)        # 3D Stokes and Polarization Parameters

In [ ]:
p_max0 = p_raw_max + spec.cf/10;  p_min0 = p_raw_min + spec.cf/10
p_max0 = np.ceil(np.log10( np.nanmax( [np.nanmax(spec.EuEu), np.nanmax(spec.EvEv), np.nanmax(spec.EwEw)] ) )*5)/5+.5
p_min0 = np.ceil(np.log10( np.nanmin( [np.nanmin(spec.EuEu), np.nanmin(spec.EvEv), np.nanmin(spec.EwEw)] ) )*5)/5-.5
p_max  = p_max0;                  p_min  = p_min0
print("conversion factor:", '{:.1f}'.format(spec.cf), "    MAX-min:", '{:.1f}'.format(p_max), '({:.1f})'.format(p_max0), '{:.1f}'.format(p_min), '({:.1f})'.format(p_min0), "   str_unit:", spec.str_unit)

### Peak

In [ ]:
print("N_samp:", data.N_samp[0], "   decimation:", juice_cdf._sample_rate(data.decimation[0]))
print("[Power]")
peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
if comp_mode==2:
    print("<PolSep>")
    peak_E_NC = np.ravel(spec.EuEu_NC[:][:]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EuEu_RC[:][:]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EuEu_LC[:][:]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EuEu: [NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), peak_f_NC, "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), peak_f_RC, "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), peak_f_LC, spec.str_unit)
    peak_E_NC = np.ravel(spec.EvEv_NC[:][:]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EvEv_RC[:][:]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EvEv_LC[:][:]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EvEv: [NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), peak_f_NC, "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), peak_f_RC, "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), peak_f_LC, spec.str_unit)
    peak_E_NC = np.ravel(spec.EwEw_NC[:][:]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EwEw_RC[:][:]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EwEw_LC[:][:]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EwEw: [NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), peak_f_NC, "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), peak_f_RC, "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), peak_f_LC, spec.str_unit)
if unit_mode==0:
    print("<RAW>")
    peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E);  print("Peak EuEu:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E);  print("Peak EvEv:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E);  print("Peak EwEw:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    if comp_mode==2:
        peak_E_NC = np.ravel(spec.EuEu_NC[:][:]); peak_f_NC = np.nanargmax(peak_E_NC)
        peak_E_RC = np.ravel(spec.EuEu_RC[:][:]); peak_f_RC = np.nanargmax(peak_E_RC)
        peak_E_LC = np.ravel(spec.EuEu_LC[:][:]); peak_f_LC = np.nanargmax(peak_E_LC)
        print("Peak EuEu: [NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:x}'.format(np.int64(peak_E_NC[peak_f_NC])), "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:x}'.format(np.int64(peak_E_RC[peak_f_RC])), "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:x}'.format(np.int64(peak_E_LC[peak_f_LC])), spec.str_unit)
        peak_E_NC = np.ravel(spec.EvEv_NC[:][:]); peak_f_NC = np.nanargmax(peak_E_NC)
        peak_E_RC = np.ravel(spec.EvEv_RC[:][:]); peak_f_RC = np.nanargmax(peak_E_RC)
        peak_E_LC = np.ravel(spec.EvEv_LC[:][:]); peak_f_LC = np.nanargmax(peak_E_LC)
        print("Peak EvEv: [NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:x}'.format(np.int64(peak_E_NC[peak_f_NC])), "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:x}'.format(np.int64(peak_E_RC[peak_f_RC])), "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:x}'.format(np.int64(peak_E_LC[peak_f_LC])), spec.str_unit)
        peak_E_NC = np.ravel(spec.EwEw_NC[:][:]); peak_f_NC = np.nanargmax(peak_E_NC)
        peak_E_RC = np.ravel(spec.EwEw_RC[:][:]); peak_f_RC = np.nanargmax(peak_E_RC)
        peak_E_LC = np.ravel(spec.EwEw_LC[:][:]); peak_f_LC = np.nanargmax(peak_E_LC)
        print("Peak EwEw: [NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:x}'.format(np.int64(peak_E_NC[peak_f_NC])), "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:x}'.format(np.int64(peak_E_RC[peak_f_RC])), "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:x}'.format(np.int64(peak_E_LC[peak_f_LC])), spec.str_unit)


In [ ]:
n_sweep = n_time1//3;  peak_E = np.ravel(spec.EuEu[n_sweep]);  peak_f = np.nanargmax(peak_E); 
print(f"[SWEEP - {n_sweep}]  Peak at", '{:.1f}'.format(freq_1d[peak_f]), "kHz", peak_f)
print("     EuEu:", '{:+.2e}'.format(np.ravel(spec.EuEu   [n_sweep])[peak_f]), spec.str_unit, "     EvEv:", '{:+.2e}'.format(np.ravel(spec.EvEv   [n_sweep])[peak_f]), spec.str_unit, "   EwEw:", '{:+.2e}'.format(np.ravel(spec.EwEw   [n_sweep])[peak_f]), spec.str_unit)
if (spec.complex[0]>0):
    print("  EuEv_re:", '{:+.2e}'.format(np.ravel(spec.EuEv_re[n_sweep])[peak_f]), spec.str_unit, "  EuEv_im:", '{:+.2e}'.format(np.ravel(spec.EuEv_im[n_sweep])[peak_f]), spec.str_unit, "  PHAuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAuv[n_sweep])[peak_f]), "  COHuv:", '{:+4.2f}'.format(np.ravel(spec.E_COHuv[n_sweep])[peak_f]), "  DoPuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoPuv[n_sweep])[peak_f]), "  DoLuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoLuv[n_sweep])[peak_f]), "  DoCuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoCuv[n_sweep])[peak_f]), "  ANGuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGuv[n_sweep])[peak_f]))
    print("  EvEw_re:", '{:+.2e}'.format(np.ravel(spec.EvEw_re[n_sweep])[peak_f]), spec.str_unit, "  EvEw_im:", '{:+.2e}'.format(np.ravel(spec.EvEw_im[n_sweep])[peak_f]), spec.str_unit, "  PHAvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAvw[n_sweep])[peak_f]), "  COHvw:", '{:+4.2f}'.format(np.ravel(spec.E_COHvw[n_sweep])[peak_f]), "  DoPvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoPvw[n_sweep])[peak_f]), "  DoLvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoLvw[n_sweep])[peak_f]), "  DoCvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoCvw[n_sweep])[peak_f]), "  ANGvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGvw[n_sweep])[peak_f]))
    print("  EwEu_re:", '{:+.2e}'.format(np.ravel(spec.EwEu_re[n_sweep])[peak_f]), spec.str_unit, "  EwEu_im:", '{:+.2e}'.format(np.ravel(spec.EwEu_im[n_sweep])[peak_f]), spec.str_unit, "  PHAwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAwu[n_sweep])[peak_f]), "  COHqu:", '{:+4.2f}'.format(np.ravel(spec.E_COHwu[n_sweep])[peak_f]), "  DoPwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoPwu[n_sweep])[peak_f]), "  DoLwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoLwu[n_sweep])[peak_f]), "  DoCwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoCwu[n_sweep])[peak_f]), "  ANGwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGwu[n_sweep])[peak_f]))
    print('Stokes-UV: {:+.2e}'.format(np.ravel(spec.E_Iuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Quv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vuv[n_sweep])[peak_f]))
    print('Stokes-VW: {:+.2e}'.format(np.ravel(spec.E_Ivw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vvw[n_sweep])[peak_f]))
    print('Stokes-WU: {:+.2e}'.format(np.ravel(spec.E_Iwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vwu[n_sweep])[peak_f]))

In [ ]:
spec.EuEu[10]

In [ ]:
n_sweep1 = n_time0//10;  n_sweep2 = n_time0//3;  n_sweep3 = n_time0-1
print(f"[First peak - {n_sweep1}]")
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if comp_mode==2:
    peak_E_NC = np.ravel(spec.EuEu_NC[n_sweep1]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EuEu_RC[n_sweep1]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EuEu_LC[n_sweep1]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EuEu:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)
    peak_E_NC = np.ravel(spec.EvEv_NC[n_sweep1]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EvEv_RC[n_sweep1]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EvEv_LC[n_sweep1]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EvEv:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)
    peak_E_NC = np.ravel(spec.EwEw_NC[n_sweep1]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EwEw_RC[n_sweep1]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EwEw_LC[n_sweep1]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EwEw:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)

print(f"[Mid   peak - {n_sweep2}]")
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if comp_mode==2:
    peak_E_NC = np.ravel(spec.EuEu_NC[n_sweep2]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EuEu_RC[n_sweep2]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EuEu_LC[n_sweep2]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EuEu:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)
    peak_E_NC = np.ravel(spec.EvEv_NC[n_sweep2]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EvEv_RC[n_sweep2]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EvEv_LC[n_sweep2]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EvEv:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)
    peak_E_NC = np.ravel(spec.EwEw_NC[n_sweep2]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EwEw_RC[n_sweep2]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EwEw_LC[n_sweep2]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EwEw:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)

print(f"[Last  peak - {n_sweep3}]")
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if comp_mode==2:
    peak_E_NC = np.ravel(spec.EuEu_NC[n_sweep3]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EuEu_RC[n_sweep3]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EuEu_LC[n_sweep3]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EuEu:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)
    peak_E_NC = np.ravel(spec.EvEv_NC[n_sweep3]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EvEv_RC[n_sweep3]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EvEv_LC[n_sweep3]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EvEv:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)
    peak_E_NC = np.ravel(spec.EwEw_NC[n_sweep3]); peak_f_NC = np.nanargmax(peak_E_NC)
    peak_E_RC = np.ravel(spec.EwEw_RC[n_sweep3]); peak_f_RC = np.nanargmax(peak_E_RC)
    peak_E_LC = np.ravel(spec.EwEw_LC[n_sweep3]); peak_f_LC = np.nanargmax(peak_E_LC)
    print("Peak EwEw:", 
          "[NC]", '{:.2e}'.format(peak_E_NC[peak_f_NC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_NC]), data.num_NC[n_sweep1][peak_f_NC], data.num_RC[n_sweep1][peak_f_NC], data.num_LC[n_sweep1][peak_f_NC],
          "[RC]", '{:.2e}'.format(peak_E_RC[peak_f_RC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_RC]), data.num_NC[n_sweep1][peak_f_RC], data.num_RC[n_sweep1][peak_f_RC], data.num_LC[n_sweep1][peak_f_RC],  
          "[LC]", '{:.2e}'.format(peak_E_LC[peak_f_LC]), '{:.2e}kHz'.format(data.frequency[n_sweep1][peak_f_LC]), data.num_NC[n_sweep1][peak_f_LC], data.num_RC[n_sweep1][peak_f_LC], data.num_LC[n_sweep1][peak_f_LC], spec.str_unit)


### Spectrum plot -- ALL

In [ ]:
fig = plt.figure(figsize=(14, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4);  ax5 = fig.add_subplot(9, 1, 5)
    ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)
else:
    ax1 = fig.add_subplot(4, 1, 1);  ax7 = fig.add_subplot(4, 1, 2);  ax8 = fig.add_subplot(4, 1, 3);  ax9 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(spec.EuEu),           '-r', linewidth=1.2, label='uu')
ax1.plot(np.ravel(spec.EvEv),           '-g', linewidth=0.8, label='vv')
ax1.plot(np.ravel(spec.EwEw),           '-b', linewidth=0.5, label='ww')
ax1.plot(np.ravel(spec.BG_Eu),          ':r', linewidth=1.2, label='BG_uu')
ax1.plot(np.ravel(spec.BG_Ev),          ':g', linewidth=0.8, label='BG_vv')
ax1.plot(np.ravel(spec.BG_Ew),          ':b', linewidth=0.5, label='BG_ww')
ax1.set_yscale('log')
ax7.plot(np.ravel(data.frequency),      '-r', linewidth=0.6, label='frequency')
ax7.plot(np.ravel(data.freq_step*10),   '-g', linewidth=0.8, label='step*10')
ax7.plot(np.ravel(data.freq_width*10),  '-b', linewidth=1.0, label='width*10')
ax8.plot(np.ravel(data.T_HF_FPGA),      ':r', label='T (HK-FPGA)')
ax8.plot(np.ravel(data.T_RWI_CH1),      ':b', label='T (RWI1)')
ax8.plot(np.ravel(data.T_RWI_CH2),      ':g', label='T (RWI2)')
ax8.plot(np.ravel(data.cal_signal*10),      '-k', label='CAL')
ax8.plot(np.ravel(data.complex*10+4),       '_r', label='comp')
ax8.plot(np.ravel(data.RFI_rejection*10+2), '_b', label='RFI')
ax9.plot(np.ravel(data.epoch),              '.')
if comp_mode > 0:    # Matrix
    ax2.plot(np.ravel(spec.EuEv_re), '-r', linewidth=1.0, label='uv_re')
    ax2.plot(np.ravel(spec.EvEw_re), '-g', linewidth=0.8, label='vw_re')
    ax2.plot(np.ravel(spec.EwEu_re), '-b', linewidth=0.6, label='wu_re')
    ax2.plot(np.ravel(spec.EuEv_im), ':r', linewidth=1.2, label='uv_im')
    ax2.plot(np.ravel(spec.EvEw_im), ':g', linewidth=1.0, label='vw_im')
    ax2.plot(np.ravel(spec.EwEu_im), ':b', linewidth=0.8, label='wu_im')
    ax3.plot(np.ravel(spec.E_DoPuv), '-r', linewidth=1.0, label='uv')
    ax3.plot(np.ravel(spec.E_DoPvw), '-g', linewidth=0.8, label='vw')
    ax3.plot(np.ravel(spec.E_DoPwu), '-b', linewidth=0.6, label='wu')
    ax4.plot(np.ravel(spec.E_DoLuv), '-r', linewidth=1.0, label='uv')
    ax4.plot(np.ravel(spec.E_DoLvw), '-g', linewidth=0.8, label='vw')
    ax4.plot(np.ravel(spec.E_DoLwu), '-b', linewidth=0.6, label='wu')
    ax5.plot(np.ravel(spec.E_DoCuv), '-r', linewidth=1.0, label='uv')
    ax5.plot(np.ravel(spec.E_DoCvw), '-g', linewidth=0.8, label='vw')
    ax5.plot(np.ravel(spec.E_DoCwu), '-b', linewidth=0.6, label='wu')
    ax6.plot(np.ravel(spec.E_ANGuv), '-r', linewidth=1.0, label='uv')
    ax6.plot(np.ravel(spec.E_ANGvw), '-g', linewidth=0.8, label='vw')
    ax6.plot(np.ravel(spec.E_ANGwu), '-b', linewidth=0.6, label='wu')

ax1.set_ylabel(spec.str_unit)
if comp_mode > 0:   # Matrix
    ax2.set_ylabel('Real & Img');     ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.')
    ax5.set_ylabel('Deg. Cir Pol.');  ax6.set_ylabel('Lin Deg.'); 
ax7.set_ylabel('frequency [kHz]');    ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time');  ax9.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);     ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
if comp_mode > 0:    # Matrix
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(np.ravel(data.EuEu)) -.5]; print(xlim); ax1.set_xlim(xlim); ax7.set_xlim(xlim)
if comp_mode > 0:    # Matrix
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))-.5]; print(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)
ylim=[10**p_min, 10**p_max];                ax1.set_ylim(ylim)
if comp_mode > 0:    # Matrix
    # ylim=[-10**p_max, 10**p_max];           ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05]; ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax5.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax6.set_ylim(ylim)
ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # Polsep - NC ALL
    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4);  ax5 = fig.add_subplot(9, 1, 5)
    ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)

    ax1.plot(np.ravel(spec.EuEu_NC),    '-r', linewidth=1.2, label='uu_NC')
    ax1.plot(np.ravel(spec.EvEv_NC),    '-g', linewidth=0.8, label='vv_NC')
    ax1.plot(np.ravel(spec.EwEw_NC),    '-b', linewidth=0.5, label='ww_NC')
    ax1.plot(np.ravel(spec.BG_Eu),          ':r', linewidth=1.2, label='BG_uu')
    ax1.plot(np.ravel(spec.BG_Ev),          ':g', linewidth=0.8, label='BG_vv')
    ax1.plot(np.ravel(spec.BG_Ew),          ':b', linewidth=0.5, label='BG_ww')
    ax1.set_yscale('log')
    ax7.plot(np.ravel(data.frequency),      '-r', linewidth=0.6, label='frequency')
    ax7.plot(np.ravel(data.freq_step*10),   '-g', linewidth=0.8, label='step*10')
    ax7.plot(np.ravel(data.freq_width*10),  '-b', linewidth=1.0, label='width*10')
    ax8.plot(np.ravel(data.T_HF_FPGA),      ':r', label='T (HK-FPGA)')
    ax8.plot(np.ravel(data.T_RWI_CH1),      ':b', label='T (RWI1)')
    ax8.plot(np.ravel(data.T_RWI_CH2),      ':g', label='T (RWI2)')
    ax8.plot(np.ravel(data.cal_signal*10),      '-k', label='CAL')
    ax8.plot(np.ravel(data.complex*10+4),       '_r', label='comp')
    ax8.plot(np.ravel(data.RFI_rejection*10+2), '_b', label='RFI')
    ax9.plot(np.ravel(data.epoch),              '.')

    ax2.plot(np.ravel(spec.EuEv_re_NC), '-r', linewidth=1.0, label='uv_re NC')
    ax2.plot(np.ravel(spec.EvEw_re_NC), '-g', linewidth=0.8, label='vw_re NC')
    ax2.plot(np.ravel(spec.EwEu_re_NC), '-b', linewidth=0.6, label='wu_re NC')
    ax2.plot(np.ravel(spec.EuEv_im_NC), ':r', linewidth=1.2, label='uv_im NC')
    ax2.plot(np.ravel(spec.EvEw_im_NC), ':g', linewidth=1.0, label='vw_im NC')
    ax2.plot(np.ravel(spec.EwEu_im_NC), ':b', linewidth=0.8, label='wu_im NC')
    ax3.plot(np.ravel(spec.E_DoPuv_NC), '-r', linewidth=1.0, label='uv NC')
    ax3.plot(np.ravel(spec.E_DoPvw_NC), '-g', linewidth=0.8, label='vw NC')
    ax3.plot(np.ravel(spec.E_DoPwu_NC), '-b', linewidth=0.6, label='wu NC')
    ax4.plot(np.ravel(spec.E_DoLuv_NC), '-r', linewidth=1.0, label='uv NC')
    ax4.plot(np.ravel(spec.E_DoLvw_NC), '-g', linewidth=0.8, label='vw NC')
    ax4.plot(np.ravel(spec.E_DoLwu_NC), '-b', linewidth=0.6, label='wu NC')
    ax5.plot(np.ravel(spec.E_DoCuv_NC), '-r', linewidth=1.0, label='uv NC')
    ax5.plot(np.ravel(spec.E_DoCvw_NC), '-g', linewidth=0.8, label='vw NC')
    ax5.plot(np.ravel(spec.E_DoCwu_NC), '-b', linewidth=0.6, label='wu NC')
    ax6.plot(np.ravel(spec.E_ANGuv_NC), '-r', linewidth=1.0, label='uv NC')
    ax6.plot(np.ravel(spec.E_ANGvw_NC), '-g', linewidth=0.8, label='vw NC')
    ax6.plot(np.ravel(spec.E_ANGwu_NC), '-b', linewidth=0.6, label='wu NC')

    ax1.set_ylabel(spec.str_unit);    ax2.set_ylabel('Real & Img');       ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.');  ax5.set_ylabel('Deg. Cir Pol.')
    ax6.set_ylabel('Lin Deg.');  ax7.set_ylabel('frequency [kHz]');  ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time');      ax9.set_xlabel(str_date)
    #
    title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
    ax1.legend(loc='upper right', fontsize=8); ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

    xlim=[-.5, len(np.ravel(data.EuEu)) -.5]; print(xlim); ax1.set_xlim(xlim); ax7.set_xlim(xlim)
    xlim=[-.5, len(np.ravel(data.epoch))-.5]; print(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
    ylim=[10**p_min, 10**p_max];                ax1.set_ylim(ylim)
    # ylim=[-10**p_max, 10**p_max];               ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05];     ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05];     ax5.set_ylim(ylim)
    ylim=[-0.05, 185.];     ax6.set_ylim(ylim)
    ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

    fig.subplots_adjust(hspace=0);  fig.show
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_raw_NC.png'
        fig.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # Polsep - RC ALL
    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4);  ax5 = fig.add_subplot(9, 1, 5)
    ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)

    ax1.plot(np.ravel(spec.EuEu_RC),    '.r', linewidth=1.2, label='uu_RC')
    ax1.plot(np.ravel(spec.EvEv_RC),    '.g', linewidth=0.8, label='vv_RC')
    ax1.plot(np.ravel(spec.EwEw_RC),    '.b', linewidth=0.5, label='ww_RC')
    ax1.plot(np.ravel(spec.BG_Eu),          ':r', linewidth=1.2, label='BG_uu')
    ax1.plot(np.ravel(spec.BG_Ev),          ':g', linewidth=0.8, label='BG_vv')
    ax1.plot(np.ravel(spec.BG_Ew),          ':b', linewidth=0.5, label='BG_ww')
    ax1.set_yscale('log')
    ax7.plot(np.ravel(data.frequency),      '-r', linewidth=0.6, label='frequency')
    ax7.plot(np.ravel(data.freq_step*10),   '-g', linewidth=0.8, label='step*10')
    ax7.plot(np.ravel(data.freq_width*10),  '-b', linewidth=1.0, label='width*10')
    ax8.plot(np.ravel(data.T_HF_FPGA),      ':r', label='T (HK-FPGA)')
    ax8.plot(np.ravel(data.T_RWI_CH1),      ':b', label='T (RWI1)')
    ax8.plot(np.ravel(data.T_RWI_CH2),      ':g', label='T (RWI2)')
    ax8.plot(np.ravel(data.cal_signal*10),      '-k', label='CAL')
    ax8.plot(np.ravel(data.complex*10+4),       '_r', label='comp')
    ax8.plot(np.ravel(data.RFI_rejection*10+2), '_b', label='RFI')
    ax9.plot(np.ravel(data.epoch),              '.')

    ax2.plot(np.ravel(spec.EuEv_re_RC), '-r', linewidth=1.0, label='uv_re RC')
    ax2.plot(np.ravel(spec.EvEw_re_RC), '-g', linewidth=0.8, label='vw_re RC')
    ax2.plot(np.ravel(spec.EwEu_re_RC), '-b', linewidth=0.6, label='wu_re RC')
    ax2.plot(np.ravel(spec.EuEv_im_RC), ':r', linewidth=1.2, label='uv_im RC')
    ax2.plot(np.ravel(spec.EvEw_im_RC), ':g', linewidth=1.0, label='vw_im RC')
    ax2.plot(np.ravel(spec.EwEu_im_RC), ':b', linewidth=0.8, label='wu_im RC')
    ax3.plot(np.ravel(spec.E_DoPuv_RC), '-r', linewidth=1.0, label='uv RC')
    ax3.plot(np.ravel(spec.E_DoPvw_RC), '-g', linewidth=0.8, label='vw RC')
    ax3.plot(np.ravel(spec.E_DoPwu_RC), '-b', linewidth=0.6, label='wu RC')
    ax4.plot(np.ravel(spec.E_DoLuv_RC), '-r', linewidth=1.0, label='uv RC')
    ax4.plot(np.ravel(spec.E_DoLvw_RC), '-g', linewidth=0.8, label='vw RC')
    ax4.plot(np.ravel(spec.E_DoLwu_RC), '-b', linewidth=0.6, label='wu RC')
    ax5.plot(np.ravel(spec.E_DoCuv_RC), '-r', linewidth=1.0, label='uv RC')
    ax5.plot(np.ravel(spec.E_DoCvw_RC), '-g', linewidth=0.8, label='vw RC')
    ax5.plot(np.ravel(spec.E_DoCwu_RC), '-b', linewidth=0.6, label='wu RC')
    ax6.plot(np.ravel(spec.E_ANGuv_RC), '-r', linewidth=1.0, label='uv RC')
    ax6.plot(np.ravel(spec.E_ANGvw_RC), '-g', linewidth=0.8, label='vw RC')
    ax6.plot(np.ravel(spec.E_ANGwu_RC), '-b', linewidth=0.6, label='wu RC')

    ax1.set_ylabel(spec.str_unit);    ax2.set_ylabel('Real & Img');       ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.');  ax5.set_ylabel('Deg. Cir Pol.')
    ax6.set_ylabel('Lin Deg.');  ax7.set_ylabel('frequency [kHz]');  ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time');      ax9.set_xlabel(str_date)
    #
    title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
    ax1.legend(loc='upper right', fontsize=8); ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

    xlim=[-.5, len(np.ravel(data.EuEu)) -.5]; print(xlim); ax1.set_xlim(xlim); ax7.set_xlim(xlim)
    xlim=[-.5, len(np.ravel(data.epoch))-.5]; print(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
    ylim=[10**p_min, 10**p_max];                ax1.set_ylim(ylim)
    # ylim=[-10**p_max, 10**p_max];               ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05];     ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05];     ax5.set_ylim(ylim)
    ylim=[-0.05, 185.];     ax6.set_ylim(ylim)
    ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

    fig.subplots_adjust(hspace=0);  fig.show
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_raw_RC.png'
        fig.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # Polsep - LC ALL
    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4);  ax5 = fig.add_subplot(9, 1, 5)
    ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)

    ax1.plot(np.ravel(spec.EuEu_LC),    '_r', linewidth=1.2, label='uu_LC')
    ax1.plot(np.ravel(spec.EvEv_LC),    '_g', linewidth=0.8, label='vv_LC')
    ax1.plot(np.ravel(spec.EwEw_LC),    '_b', linewidth=0.5, label='ww_LC')
    ax1.plot(np.ravel(spec.BG_Eu),          ':r', linewidth=1.2, label='BG_uu')
    ax1.plot(np.ravel(spec.BG_Ev),          ':g', linewidth=0.8, label='BG_vv')
    ax1.plot(np.ravel(spec.BG_Ew),          ':b', linewidth=0.5, label='BG_ww')
    ax1.set_yscale('log')
    ax7.plot(np.ravel(data.frequency),      '-r', linewidth=0.6, label='frequency')
    ax7.plot(np.ravel(data.freq_step*10),   '-g', linewidth=0.8, label='step*10')
    ax7.plot(np.ravel(data.freq_width*10),  '-b', linewidth=1.0, label='width*10')
    ax8.plot(np.ravel(data.T_HF_FPGA),      ':r', label='T (HK-FPGA)')
    ax8.plot(np.ravel(data.T_RWI_CH1),      ':b', label='T (RWI1)')
    ax8.plot(np.ravel(data.T_RWI_CH2),      ':g', label='T (RWI2)')
    ax8.plot(np.ravel(data.cal_signal*10),      '-k', label='CAL')
    ax8.plot(np.ravel(data.complex*10+4),       '_r', label='comp')
    ax8.plot(np.ravel(data.RFI_rejection*10+2), '_b', label='RFI')
    ax9.plot(np.ravel(data.epoch),              '.')

    ax2.plot(np.ravel(spec.EuEv_re_LC), '-r', linewidth=1.0, label='uv_re LC')
    ax2.plot(np.ravel(spec.EvEw_re_LC), '-g', linewidth=0.8, label='vw_re LC')
    ax2.plot(np.ravel(spec.EwEu_re_LC), '-b', linewidth=0.6, label='wu_re LC')
    ax2.plot(np.ravel(spec.EuEv_im_LC), ':r', linewidth=1.2, label='uv_im LC')
    ax2.plot(np.ravel(spec.EvEw_im_LC), ':g', linewidth=1.0, label='vw_im LC')
    ax2.plot(np.ravel(spec.EwEu_im_LC), ':b', linewidth=0.8, label='wu_im LC')
    ax3.plot(np.ravel(spec.E_DoPuv_LC), '-r', linewidth=1.0, label='uv LC')
    ax3.plot(np.ravel(spec.E_DoPvw_LC), '-g', linewidth=0.8, label='vw LC')
    ax3.plot(np.ravel(spec.E_DoPwu_LC), '-b', linewidth=0.6, label='wu LC')
    ax4.plot(np.ravel(spec.E_DoLuv_LC), '-r', linewidth=1.0, label='uv LC')
    ax4.plot(np.ravel(spec.E_DoLvw_LC), '-g', linewidth=0.8, label='vw LC')
    ax4.plot(np.ravel(spec.E_DoLwu_LC), '-b', linewidth=0.6, label='wu LC')
    ax5.plot(np.ravel(spec.E_DoCuv_LC), '-r', linewidth=1.0, label='uv LC')
    ax5.plot(np.ravel(spec.E_DoCvw_LC), '-g', linewidth=0.8, label='vw LC')
    ax5.plot(np.ravel(spec.E_DoCwu_LC), '-b', linewidth=0.6, label='wu LC')
    ax6.plot(np.ravel(spec.E_ANGuv_LC), '-r', linewidth=1.0, label='uv LC')
    ax6.plot(np.ravel(spec.E_ANGvw_LC), '-g', linewidth=0.8, label='vw LC')
    ax6.plot(np.ravel(spec.E_ANGwu_LC), '-b', linewidth=0.6, label='wu LC')
    
    ax1.set_ylabel(spec.str_unit);    ax2.set_ylabel('Real & Img');       ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.');  ax5.set_ylabel('Deg. Cir Pol.')
    ax6.set_ylabel('Lin Deg.');  ax7.set_ylabel('frequency [kHz]');  ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time');      ax9.set_xlabel(str_date)
    #
    title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
    ax1.legend(loc='upper right', fontsize=8); ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

    xlim=[-.5, len(np.ravel(data.EuEu)) -.5]; print(xlim); ax1.set_xlim(xlim); ax7.set_xlim(xlim)
    xlim=[-.5, len(np.ravel(data.epoch))-.5]; print(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
    ylim=[10**p_min, 10**p_max];                ax1.set_ylim(ylim)
    #ylim=[-10**p_max, 10**p_max];               ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05];     ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05];     ax5.set_ylim(ylim)
    ylim=[-0.05, 185.];     ax6.set_ylim(ylim)
    ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

    fig.subplots_adjust(hspace=0);  fig.show
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_raw_LC.png'
        fig.savefig(png_fname)

### Spectrum plot -- Single

In [ ]:
n_time = n_time1//2;  print("EuEu:", spec.EuEu[n_time][0])
fig = plt.figure(figsize=(14, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(7, 1, 1);  ax2 = fig.add_subplot(7, 1, 2);  ax3 = fig.add_subplot(7, 1, 3);  ax4 = fig.add_subplot(7, 1, 4);  ax5 = fig.add_subplot(7, 1, 5)
    ax6 = fig.add_subplot(7, 1, 6);  ax7 = fig.add_subplot(7, 1, 7)
else:
    ax1 = fig.add_subplot(2, 1, 1);  ax7 = fig.add_subplot(2, 1, 2)

ax1.plot(np.ravel(spec.EuEu[n_time]),       '-r', linewidth=1.2, label='uu')
ax1.plot(np.ravel(spec.EvEv[n_time]),       '-g', linewidth=0.8, label='vv')
ax1.plot(np.ravel(spec.EwEw[n_time]),       '-b', linewidth=0.5, label='ww')
ax1.plot(np.ravel(spec.BG_Eu[n_time]),      ':r', linewidth=1.2, label='BG_uu')
ax1.plot(np.ravel(spec.BG_Ev[n_time]),      ':g', linewidth=0.8, label='BG_vv')
ax1.plot(np.ravel(spec.BG_Ew[n_time]),      ':b', linewidth=0.5, label='BG_ww')
ax1.set_yscale('log')
ax7.plot(np.ravel(data.frequency[n_time]),      '-r', linewidth=0.6, label='frequency')
ax7.plot(np.ravel(data.freq_step[n_time]*10),   '-g', linewidth=0.8, label='step*10')
ax7.plot(np.ravel(data.freq_width[n_time]*10),  '-b', linewidth=1.0, label='width*10')
if comp_mode > 0:    # Matrix
    ax2.plot(np.ravel(spec.EuEv_re[n_time]), '-r', linewidth=1.0, label='uv_re')
    ax2.plot(np.ravel(spec.EvEw_re[n_time]), '-g', linewidth=0.8, label='vw_re')
    ax2.plot(np.ravel(spec.EwEu_re[n_time]), '-b', linewidth=0.6, label='wu_re')
    ax2.plot(np.ravel(spec.EuEv_im[n_time]), ':r', linewidth=1.2, label='uv_im')
    ax2.plot(np.ravel(spec.EvEw_im[n_time]), ':g', linewidth=1.0, label='vw_im')
    ax2.plot(np.ravel(spec.EwEu_im[n_time]), ':b', linewidth=0.8, label='wu_im')
    ax3.plot(np.ravel(spec.E_DoPuv[n_time]), '-r', linewidth=1.0, label='uv')
    ax3.plot(np.ravel(spec.E_DoPvw[n_time]), '-g', linewidth=0.8, label='vw')
    ax3.plot(np.ravel(spec.E_DoPwu[n_time]), '-b', linewidth=0.6, label='wu')
    ax4.plot(np.ravel(spec.E_DoLuv[n_time]), '-r', linewidth=1.0, label='uv')
    ax4.plot(np.ravel(spec.E_DoLvw[n_time]), '-g', linewidth=0.8, label='vw')
    ax4.plot(np.ravel(spec.E_DoLwu[n_time]), '-b', linewidth=0.6, label='wu')
    ax5.plot(np.ravel(spec.E_DoCuv[n_time]), '-r', linewidth=1.0, label='uv')
    ax5.plot(np.ravel(spec.E_DoCvw[n_time]), '-g', linewidth=0.8, label='vw')
    ax5.plot(np.ravel(spec.E_DoCwu[n_time]), '-b', linewidth=0.6, label='wu')
    ax6.plot(np.ravel(spec.E_ANGuv[n_time]), '-r', linewidth=1.0, label='uv')
    ax6.plot(np.ravel(spec.E_ANGvw[n_time]), '-g', linewidth=0.8, label='vw')
    ax6.plot(np.ravel(spec.E_ANGwu[n_time]), '-b', linewidth=0.6, label='wu')

ax1.set_ylabel(spec.str_unit)
if comp_mode > 0:    # Matrix
    ax2.set_ylabel('Real & Img');     ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.')
    ax5.set_ylabel('Deg. Cir Pol.');  ax6.set_ylabel('Lin Deg.')
ax7.set_ylabel('frequency [kHz]');    ax7.set_xlabel(data.epoch[n_time].strftime('%Y/%m/%d %R:%S') + '  [in ' + str_date + ']')
#
title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);     ax7.legend(loc='upper right', fontsize=8); 
if comp_mode > 0:    # Matrix
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(data.EuEu[n_time]) -.5];   ax1.set_xlim(xlim); ax7.set_xlim(xlim)
if comp_mode > 0:    # Matrix
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))-.5]; ax8.set_xlim(xlim); ax9.set_xlim(xlim)
# ylim=[10**p_raw_min, 10**p_raw_max];  ax1.set_ylim(ylim)
if comp_mode > 0:    # Matrix
    # ylim=[-10**p_max, 10**p_max];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05]; ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax5.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax6.set_ylim(ylim)
ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw2.png'
    fig.savefig(png_fname)

In [ ]:
if comp_mode == 2:  # Polsep NC Single
    n_time = n_time1//2;  print("EuEu_NC:", spec.EuEu_NC[n_time][0])
    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(7, 1, 1);  ax2 = fig.add_subplot(7, 1, 2);  ax3 = fig.add_subplot(7, 1, 3);  ax4 = fig.add_subplot(7, 1, 4);  ax5 = fig.add_subplot(7, 1, 5)
    ax6 = fig.add_subplot(7, 1, 6);  ax7 = fig.add_subplot(7, 1, 7)
    ax1.plot(np.ravel(spec.EuEu_NC[n_time]),    '-r', linewidth=1.2, label='uu_NC')
    ax1.plot(np.ravel(spec.EvEv_NC[n_time]),    '-g', linewidth=0.8, label='vv_NC')
    ax1.plot(np.ravel(spec.EwEw_NC[n_time]),    '-b', linewidth=0.5, label='ww_NC')
    ax1.plot(np.ravel(spec.BG_Eu[n_time]),          ':r', linewidth=1.2, label='BG_uu')
    ax1.plot(np.ravel(spec.BG_Ev[n_time]),          ':g', linewidth=0.8, label='BG_vv')
    ax1.plot(np.ravel(spec.BG_Ew[n_time]),          ':b', linewidth=0.5, label='BG_ww')
    ax1.set_yscale('log')
    ax7.plot(np.ravel(data.frequency[n_time]),      '-r', linewidth=0.6, label='frequency')
    ax7.plot(np.ravel(data.freq_step[n_time]*10),   '-g', linewidth=0.8, label='step*10')
    ax7.plot(np.ravel(data.freq_width[n_time]*10),  '-b', linewidth=1.0, label='width*10')

    ax2.plot(np.ravel(spec.EuEv_re_NC[n_time]), '-r', linewidth=1.0, label='uv_re NC')
    ax2.plot(np.ravel(spec.EvEw_re_NC[n_time]), '-g', linewidth=0.8, label='vw_re NC')
    ax2.plot(np.ravel(spec.EwEu_re_NC[n_time]), '-b', linewidth=0.6, label='wu_re NC')
    ax2.plot(np.ravel(spec.EuEv_im_NC[n_time]), ':r', linewidth=1.2, label='uv_im NC')
    ax2.plot(np.ravel(spec.EvEw_im_NC[n_time]), ':g', linewidth=1.0, label='vw_im NC')
    ax2.plot(np.ravel(spec.EwEu_im_NC[n_time]), ':b', linewidth=0.8, label='wu_im NC')
    ax3.plot(np.ravel(spec.E_DoPuv_NC[n_time]), '-r', linewidth=1.0, label='uv NC')
    ax3.plot(np.ravel(spec.E_DoPvw_NC[n_time]), '-g', linewidth=0.8, label='vw NC')
    ax3.plot(np.ravel(spec.E_DoPwu_NC[n_time]), '-b', linewidth=0.6, label='wu NC')
    ax4.plot(np.ravel(spec.E_DoLuv_NC[n_time]), '-r', linewidth=1.0, label='uv NC')
    ax4.plot(np.ravel(spec.E_DoLvw_NC[n_time]), '-g', linewidth=0.8, label='vw NC')
    ax4.plot(np.ravel(spec.E_DoLwu_NC[n_time]), '-b', linewidth=0.6, label='wu NC')
    ax5.plot(np.ravel(spec.E_DoCuv_NC[n_time]), '-r', linewidth=1.0, label='uv NC')
    ax5.plot(np.ravel(spec.E_DoCvw_NC[n_time]), '-g', linewidth=0.8, label='vw NC')
    ax5.plot(np.ravel(spec.E_DoCwu_NC[n_time]), '-b', linewidth=0.6, label='wu NC')
    ax6.plot(np.ravel(spec.E_ANGuv_NC[n_time]), '-r', linewidth=1.0, label='uv NC')
    ax6.plot(np.ravel(spec.E_ANGvw_NC[n_time]), '-g', linewidth=0.8, label='vw NC')
    ax6.plot(np.ravel(spec.E_ANGwu_NC[n_time]), '-b', linewidth=0.6, label='wu NC')

    ax1.set_ylabel(spec.str_unit);    ax2.set_ylabel('Real & Img');       ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.');  ax5.set_ylabel('Deg. Cir Pol.')
    ax6.set_ylabel('Lin Deg.');  ax7.set_ylabel('frequency [kHz]');  ax7.set_xlabel(data.epoch[n_time].strftime('%Y/%m/%d %R:%S') + '  [in ' + str_date + ']')
    #
    title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
    ax1.legend(loc='upper right', fontsize=8); ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

    xlim=[-.5, len(data.EuEu_NC[n_time]) -.5]; ax1.set_xlim(xlim); ax7.set_xlim(xlim)
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
    xlim=[-.5, len(np.ravel(data.epoch))-.5];  ax8.set_xlim(xlim); ax9.set_xlim(xlim)
    # ylim=[10**p_raw_min, 10**p_raw_max];  ax1.set_ylim(ylim)
    # ylim=[-10**p_max, 10**p_max];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05];     ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05];     ax5.set_ylim(ylim)
    ylim=[-0.05, 185.];     ax6.set_ylim(ylim)
    ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

    fig.subplots_adjust(hspace=0);  fig.show
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_raw2-NC.png'
        fig.savefig(png_fname)

In [ ]:
if comp_mode == 2:  # Polsep RC Single
    n_time = n_time1//2;  print("EuEu_RC:", spec.EuEu_RC[n_time][0])
    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(7, 1, 1);  ax2 = fig.add_subplot(7, 1, 2);  ax3 = fig.add_subplot(7, 1, 3);  ax4 = fig.add_subplot(7, 1, 4);  ax5 = fig.add_subplot(7, 1, 5)
    ax6 = fig.add_subplot(7, 1, 6);  ax7 = fig.add_subplot(7, 1, 7)
    ax1.plot(np.ravel(spec.EuEu_RC[n_time]),    '-r', linewidth=1.2, label='uu_RC')
    ax1.plot(np.ravel(spec.EvEv_RC[n_time]),    '-g', linewidth=0.8, label='vv_RC')
    ax1.plot(np.ravel(spec.EwEw_RC[n_time]),    '-b', linewidth=0.5, label='ww_RC')
    ax1.plot(np.ravel(spec.BG_Eu[n_time]),      ':r', linewidth=1.2, label='BG_uu')
    ax1.plot(np.ravel(spec.BG_Ev[n_time]),      ':g', linewidth=0.8, label='BG_vv')
    ax1.plot(np.ravel(spec.BG_Ew[n_time]),      ':b', linewidth=0.5, label='BG_ww')
    ax1.set_yscale('log')
    ax7.plot(np.ravel(data.frequency[n_time]),      '-r', linewidth=0.6, label='frequency')
    ax7.plot(np.ravel(data.freq_step[n_time]*10),   '-g', linewidth=0.8, label='step*10')
    ax7.plot(np.ravel(data.freq_width[n_time]*10),  '-b', linewidth=1.0, label='width*10')

    ax2.plot(np.ravel(spec.EuEv_re_RC[n_time]), '-r', linewidth=1.0, label='uv_re RC')
    ax2.plot(np.ravel(spec.EvEw_re_RC[n_time]), '-g', linewidth=0.8, label='vw_re RC')
    ax2.plot(np.ravel(spec.EwEu_re_RC[n_time]), '-b', linewidth=0.6, label='wu_re RC')
    ax2.plot(np.ravel(spec.EuEv_im_RC[n_time]), ':r', linewidth=1.2, label='uv_im RC')
    ax2.plot(np.ravel(spec.EvEw_im_RC[n_time]), ':g', linewidth=1.0, label='vw_im RC')
    ax2.plot(np.ravel(spec.EwEu_im_RC[n_time]), ':b', linewidth=0.8, label='wu_im RC')
    ax3.plot(np.ravel(spec.E_DoPuv_RC[n_time]), '-r', linewidth=1.0, label='uv RC')
    ax3.plot(np.ravel(spec.E_DoPvw_RC[n_time]), '-g', linewidth=0.8, label='vw RC')
    ax3.plot(np.ravel(spec.E_DoPwu_RC[n_time]), '-b', linewidth=0.6, label='wu RC')
    ax4.plot(np.ravel(spec.E_DoLuv_RC[n_time]), '-r', linewidth=1.0, label='uv RC')
    ax4.plot(np.ravel(spec.E_DoLvw_RC[n_time]), '-g', linewidth=0.8, label='vw RC')
    ax4.plot(np.ravel(spec.E_DoLwu_RC[n_time]), '-b', linewidth=0.6, label='wu RC')
    ax5.plot(np.ravel(spec.E_DoCuv_RC[n_time]), '-r', linewidth=1.0, label='uv RC')
    ax5.plot(np.ravel(spec.E_DoCvw_RC[n_time]), '-g', linewidth=0.8, label='vw RC')
    ax5.plot(np.ravel(spec.E_DoCwu_RC[n_time]), '-b', linewidth=0.6, label='wu RC')
    ax6.plot(np.ravel(spec.E_ANGuv_RC[n_time]), '-r', linewidth=1.0, label='uv RC')
    ax6.plot(np.ravel(spec.E_ANGvw_RC[n_time]), '-g', linewidth=0.8, label='vw RC')
    ax6.plot(np.ravel(spec.E_ANGwu_RC[n_time]), '-b', linewidth=0.6, label='wu RC')

    ax1.set_ylabel(spec.str_unit);    ax2.set_ylabel('Real & Img');       ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.');  ax5.set_ylabel('Deg. Cir Pol.')
    ax6.set_ylabel('Lin Deg.');  ax7.set_ylabel('frequency [kHz]');  ax7.set_xlabel(data.epoch[n_time].strftime('%Y/%m/%d %R:%S') + '  [in ' + str_date + ']')
    #
    title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
    ax1.legend(loc='upper right', fontsize=8); ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

    xlim=[-.5, len(data.EuEu_RC[n_time]) -.5]; ax1.set_xlim(xlim); ax7.set_xlim(xlim)
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
    xlim=[-.5, len(np.ravel(data.epoch))-.5];  ax8.set_xlim(xlim); ax9.set_xlim(xlim)
    # ylim=[10**p_raw_min, 10**p_raw_max];  ax1.set_ylim(ylim)
    # ylim=[-10**p_max, 10**p_max];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05];     ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05];     ax5.set_ylim(ylim)
    ylim=[-0.05, 185.];     ax6.set_ylim(ylim)
    ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

    fig.subplots_adjust(hspace=0);  fig.show
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_raw2-RC.png'
        fig.savefig(png_fname)

In [ ]:
if comp_mode == 2:  # Polsep LC Single
    n_time = n_time1//2;  print("EuEu_LC:", spec.EuEu_LC[n_time][0])
    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(7, 1, 1);  ax2 = fig.add_subplot(7, 1, 2);  ax3 = fig.add_subplot(7, 1, 3);  ax4 = fig.add_subplot(7, 1, 4);  ax5 = fig.add_subplot(7, 1, 5)
    ax6 = fig.add_subplot(7, 1, 6);  ax7 = fig.add_subplot(7, 1, 7)
    ax1.plot(np.ravel(spec.EuEu_LC[n_time]),    '-r', linewidth=1.2, label='uu_LC')
    ax1.plot(np.ravel(spec.EvEv_LC[n_time]),    '-g', linewidth=0.8, label='vv_LC')
    ax1.plot(np.ravel(spec.EwEw_LC[n_time]),    '-b', linewidth=0.5, label='ww_LC')
    ax1.plot(np.ravel(spec.BG_Eu[n_time]),      ':r', linewidth=1.2, label='BG_uu')
    ax1.plot(np.ravel(spec.BG_Ev[n_time]),      ':g', linewidth=0.8, label='BG_vv')
    ax1.plot(np.ravel(spec.BG_Ew[n_time]),      ':b', linewidth=0.5, label='BG_ww')
    ax1.set_yscale('log')
    ax7.plot(np.ravel(data.frequency[n_time]),      '-r', linewidth=0.6, label='frequency')
    ax7.plot(np.ravel(data.freq_step[n_time]*10),   '-g', linewidth=0.8, label='step*10')
    ax7.plot(np.ravel(data.freq_width[n_time]*10),  '-b', linewidth=1.0, label='width*10')

    ax2.plot(np.ravel(spec.EuEv_re_LC[n_time]), '-r', linewidth=1.0, label='uv_re LC')
    ax2.plot(np.ravel(spec.EvEw_re_LC[n_time]), '-g', linewidth=0.8, label='vw_re LC')
    ax2.plot(np.ravel(spec.EwEu_re_LC[n_time]), '-b', linewidth=0.6, label='wu_re LC')
    ax2.plot(np.ravel(spec.EuEv_im_LC[n_time]), ':r', linewidth=1.2, label='uv_im LC')
    ax2.plot(np.ravel(spec.EvEw_im_LC[n_time]), ':g', linewidth=1.0, label='vw_im LC')
    ax2.plot(np.ravel(spec.EwEu_im_LC[n_time]), ':b', linewidth=0.8, label='wu_im LC')
    ax3.plot(np.ravel(spec.E_DoPuv_LC[n_time]), '-r', linewidth=1.0, label='uv LC')
    ax3.plot(np.ravel(spec.E_DoPvw_LC[n_time]), '-g', linewidth=0.8, label='vw LC')
    ax3.plot(np.ravel(spec.E_DoPwu_LC[n_time]), '-b', linewidth=0.6, label='wu LC')
    ax4.plot(np.ravel(spec.E_DoLuv_LC[n_time]), '-r', linewidth=1.0, label='uv LC')
    ax4.plot(np.ravel(spec.E_DoLvw_LC[n_time]), '-g', linewidth=0.8, label='vw LC')
    ax4.plot(np.ravel(spec.E_DoLwu_LC[n_time]), '-b', linewidth=0.6, label='wu LC')
    ax5.plot(np.ravel(spec.E_DoCuv_LC[n_time]), '-r', linewidth=1.0, label='uv LC')
    ax5.plot(np.ravel(spec.E_DoCvw_LC[n_time]), '-g', linewidth=0.8, label='vw LC')
    ax5.plot(np.ravel(spec.E_DoCwu_LC[n_time]), '-b', linewidth=0.6, label='wu LC')
    ax6.plot(np.ravel(spec.E_ANGuv_LC[n_time]), '-r', linewidth=1.0, label='uv LC')
    ax6.plot(np.ravel(spec.E_ANGvw_LC[n_time]), '-g', linewidth=0.8, label='vw LC')
    ax6.plot(np.ravel(spec.E_ANGwu_LC[n_time]), '-b', linewidth=0.6, label='wu LC')

    ax1.set_ylabel(spec.str_unit);    ax2.set_ylabel('Real & Img');       ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.');  ax5.set_ylabel('Deg. Cir Pol.')
    ax6.set_ylabel('Lin Deg.');  ax7.set_ylabel('frequency [kHz]');  ax7.set_xlabel(data.epoch[n_time].strftime('%Y/%m/%d %R:%S') + '  [in ' + str_date + ']')
    #
    title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
    ax1.legend(loc='upper right', fontsize=8); ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

    xlim=[-.5, len(data.EuEu_LC[n_time]) -.5]; ax1.set_xlim(xlim); ax7.set_xlim(xlim)
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
    xlim=[-.5, len(np.ravel(data.epoch))-.5];  ax8.set_xlim(xlim); ax9.set_xlim(xlim)
    # ylim=[10**p_raw_min, 10**p_raw_max];  ax1.set_ylim(ylim)
    # ylim=[-10**p_max, 10**p_max];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05];     ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05];     ax5.set_ylim(ylim)
    ylim=[-0.05, 185.];     ax6.set_ylim(ylim)
    ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

    fig.subplots_adjust(hspace=0);  fig.show
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_raw2-LC.png'
        fig.savefig(png_fname)

In [ ]:
print("Complex:", data.complex)
print("CAL_sig:", data.cal_signal)
print("POL_SW :", data.Pol_sep_SW)
print("POL_thr:", data.Pol_sep_thres)

### FT Plot data

In [ ]:
# Normal spectrum plots
spec.EE    = spec.EuEu + spec.EvEv + spec.EwEw
EuEu_2d    = spec.EuEu.transpose();     EuEu_med    = np.nanmedian(spec.EuEu,    axis=0)
EvEv_2d    = spec.EvEv.transpose();     EvEv_med    = np.nanmedian(spec.EvEv,    axis=0)
EwEw_2d    = spec.EwEw.transpose();     EwEw_med    = np.nanmedian(spec.EwEw,    axis=0)
EE_2d      = spec.EE.transpose();       EE_med      = np.nanmedian(spec.EE,      axis=0)
E_DoPuv_2d = spec.E_DoLuv.transpose();  E_DoPuv_med = np.nanmedian(spec.E_DoPuv, axis=0)
E_DoPvw_2d = spec.E_DoLvw.transpose();  E_DoPvw_med = np.nanmedian(spec.E_DoPvw, axis=0)
E_DoPwu_2d = spec.E_DoLwu.transpose();  E_DoPwu_med = np.nanmedian(spec.E_DoPwu, axis=0)
E_DoLuv_2d = spec.E_DoLuv.transpose();  E_DoLuv_med = np.nanmedian(spec.E_DoLuv, axis=0)
E_DoLvw_2d = spec.E_DoLvw.transpose();  E_DoLvw_med = np.nanmedian(spec.E_DoLvw, axis=0)
E_DoLwu_2d = spec.E_DoLwu.transpose();  E_DoLwu_med = np.nanmedian(spec.E_DoLwu, axis=0)
E_DoCuv_2d = spec.E_DoCuv.transpose();  E_DoCuv_med = np.nanmedian(spec.E_DoCuv, axis=0)
E_DoCvw_2d = spec.E_DoCvw.transpose();  E_DoCvw_med = np.nanmedian(spec.E_DoCvw, axis=0)
E_DoCwu_2d = spec.E_DoCwu.transpose();  E_DoCwu_med = np.nanmedian(spec.E_DoCwu, axis=0)
E_ANGuv_2d = spec.E_ANGuv.transpose();  E_ANGuv_med = np.nanmedian(spec.E_ANGuv, axis=0)
E_ANGvw_2d = spec.E_ANGvw.transpose();  E_ANGvw_med = np.nanmedian(spec.E_ANGvw, axis=0)
E_ANGwu_2d = spec.E_ANGwu.transpose();  E_ANGwu_med = np.nanmedian(spec.E_ANGwu, axis=0)
E_COHuv_2d = spec.E_ANGwu.transpose();  E_COHuv_med = np.nanmedian(spec.E_COHuv, axis=0)
E_COHvw_2d = spec.E_ANGwu.transpose();  E_COHvw_med = np.nanmedian(spec.E_COHvw, axis=0)
E_COHwu_2d = spec.E_ANGwu.transpose();  E_COHwu_med = np.nanmedian(spec.E_COHwu, axis=0)
E_PHAuv_2d = spec.E_ANGwu.transpose();  E_PHAuv_med = np.nanmedian(spec.E_PHAuv, axis=0)
E_PHAvw_2d = spec.E_ANGwu.transpose();  E_PHAvw_med = np.nanmedian(spec.E_PHAvw, axis=0)
E_PHAwu_2d = spec.E_ANGwu.transpose();  E_PHAwu_med = np.nanmedian(spec.E_PHAwu, axis=0)

if comp_mode == 2:  # Polsep RC Single
    # Polsep spectrum plots
    spec.EE_NC    = spec.EuEu_NC + spec.EvEv_NC + spec.EwEw_NC
    EuEu_NC_2d    = spec.EuEu_NC.transpose();     EuEu_NC_med = np.nanmedian(spec.EuEu_NC,    axis=0)
    EvEv_NC_2d    = spec.EvEv_NC.transpose();     EvEv_NC_med = np.nanmedian(spec.EvEv_NC,    axis=0)
    EwEw_NC_2d    = spec.EwEw_NC.transpose();     EwEw_NC_med = np.nanmedian(spec.EwEw_NC,    axis=0)
    EE_NC_2d      = spec.EE_NC.transpose();       EE_NC_med   = np.nanmedian(spec.EE_NC,      axis=0)
    E_DoPuv_NC_2d = spec.E_DoLuv_NC.transpose();  E_DoPuv_NC_med = np.nanmedian(spec.E_DoPuv_NC, axis=0)
    E_DoPvw_NC_2d = spec.E_DoLvw_NC.transpose();  E_DoPvw_NC_med = np.nanmedian(spec.E_DoPvw_NC, axis=0)
    E_DoPwu_NC_2d = spec.E_DoLwu_NC.transpose();  E_DoPwu_NC_med = np.nanmedian(spec.E_DoPwu_NC, axis=0)
    E_DoLuv_NC_2d = spec.E_DoLuv_NC.transpose();  E_DoLuv_NC_med = np.nanmedian(spec.E_DoLuv_NC, axis=0)
    E_DoLvw_NC_2d = spec.E_DoLvw_NC.transpose();  E_DoLvw_NC_med = np.nanmedian(spec.E_DoLvw_NC, axis=0)
    E_DoLwu_NC_2d = spec.E_DoLwu_NC.transpose();  E_DoLwu_NC_med = np.nanmedian(spec.E_DoLwu_NC, axis=0)
    E_DoCuv_NC_2d = spec.E_DoCuv_NC.transpose();  E_DoCuv_NC_med = np.nanmedian(spec.E_DoCuv_NC, axis=0)
    E_DoCvw_NC_2d = spec.E_DoCvw_NC.transpose();  E_DoCvw_NC_med = np.nanmedian(spec.E_DoCvw_NC, axis=0)
    E_DoCwu_NC_2d = spec.E_DoCwu_NC.transpose();  E_DoCwu_NC_med = np.nanmedian(spec.E_DoCwu_NC, axis=0)
    E_ANGuv_NC_2d = spec.E_ANGuv_NC.transpose();  E_ANGuv_NC_med = np.nanmedian(spec.E_ANGuv_NC, axis=0)
    E_ANGvw_NC_2d = spec.E_ANGvw_NC.transpose();  E_ANGvw_NC_med = np.nanmedian(spec.E_ANGvw_NC, axis=0)
    E_ANGwu_NC_2d = spec.E_ANGwu_NC.transpose();  E_ANGwu_NC_med = np.nanmedian(spec.E_ANGwu_NC, axis=0)
    E_COHuv_NC_2d = spec.E_ANGwu_NC.transpose();  E_COHuv_NC_med = np.nanmedian(spec.E_COHuv_NC, axis=0)
    E_COHvw_NC_2d = spec.E_ANGwu_NC.transpose();  E_COHvw_NC_med = np.nanmedian(spec.E_COHvw_NC, axis=0)
    E_COHwu_NC_2d = spec.E_ANGwu_NC.transpose();  E_COHwu_NC_med = np.nanmedian(spec.E_COHwu_NC, axis=0)
    E_PHAuv_NC_2d = spec.E_ANGwu_NC.transpose();  E_PHAuv_NC_med = np.nanmedian(spec.E_PHAuv_NC, axis=0)
    E_PHAvw_NC_2d = spec.E_ANGwu_NC.transpose();  E_PHAvw_NC_med = np.nanmedian(spec.E_PHAvw_NC, axis=0)
    E_PHAwu_NC_2d = spec.E_ANGwu_NC.transpose();  E_PHAwu_NC_med = np.nanmedian(spec.E_PHAwu_NC, axis=0)

    spec.EE_RC    = spec.EuEu_RC + spec.EvEv_RC + spec.EwEw_RC
    EuEu_RC_2d    = spec.EuEu_RC.transpose();     EuEu_RC_med = np.nanmedian(spec.EuEu_RC,    axis=0)
    EvEv_RC_2d    = spec.EvEv_RC.transpose();     EvEv_RC_med = np.nanmedian(spec.EvEv_RC,    axis=0)
    EwEw_RC_2d    = spec.EwEw_RC.transpose();     EwEw_RC_med = np.nanmedian(spec.EwEw_RC,    axis=0)
    EE_RC_2d      = spec.EE_RC.transpose();       EE_RC_med   = np.nanmedian(spec.EE_RC,      axis=0)
    E_DoPuv_RC_2d = spec.E_DoLuv_RC.transpose();  E_DoPuv_RC_med = np.nanmedian(spec.E_DoPuv_RC, axis=0)
    E_DoPvw_RC_2d = spec.E_DoLvw_RC.transpose();  E_DoPvw_RC_med = np.nanmedian(spec.E_DoPvw_RC, axis=0)
    E_DoPwu_RC_2d = spec.E_DoLwu_RC.transpose();  E_DoPwu_RC_med = np.nanmedian(spec.E_DoPwu_RC, axis=0)
    E_DoLuv_RC_2d = spec.E_DoLuv_RC.transpose();  E_DoLuv_RC_med = np.nanmedian(spec.E_DoLuv_RC, axis=0)
    E_DoLvw_RC_2d = spec.E_DoLvw_RC.transpose();  E_DoLvw_RC_med = np.nanmedian(spec.E_DoLvw_RC, axis=0)
    E_DoLwu_RC_2d = spec.E_DoLwu_RC.transpose();  E_DoLwu_RC_med = np.nanmedian(spec.E_DoLwu_RC, axis=0)
    E_DoCuv_RC_2d = spec.E_DoCuv_RC.transpose();  E_DoCuv_RC_med = np.nanmedian(spec.E_DoCuv_RC, axis=0)
    E_DoCvw_RC_2d = spec.E_DoCvw_RC.transpose();  E_DoCvw_RC_med = np.nanmedian(spec.E_DoCvw_RC, axis=0)
    E_DoCwu_RC_2d = spec.E_DoCwu_RC.transpose();  E_DoCwu_RC_med = np.nanmedian(spec.E_DoCwu_RC, axis=0)
    E_ANGuv_RC_2d = spec.E_ANGuv_RC.transpose();  E_ANGuv_RC_med = np.nanmedian(spec.E_ANGuv_RC, axis=0)
    E_ANGvw_RC_2d = spec.E_ANGvw_RC.transpose();  E_ANGvw_RC_med = np.nanmedian(spec.E_ANGvw_RC, axis=0)
    E_ANGwu_RC_2d = spec.E_ANGwu_RC.transpose();  E_ANGwu_RC_med = np.nanmedian(spec.E_ANGwu_RC, axis=0)
    E_COHuv_RC_2d = spec.E_ANGwu_RC.transpose();  E_COHuv_RC_med = np.nanmedian(spec.E_COHuv_RC, axis=0)
    E_COHvw_RC_2d = spec.E_ANGwu_RC.transpose();  E_COHvw_RC_med = np.nanmedian(spec.E_COHvw_RC, axis=0)
    E_COHwu_RC_2d = spec.E_ANGwu_RC.transpose();  E_COHwu_RC_med = np.nanmedian(spec.E_COHwu_RC, axis=0)
    E_PHAuv_RC_2d = spec.E_ANGwu_RC.transpose();  E_PHAuv_RC_med = np.nanmedian(spec.E_PHAuv_RC, axis=0)
    E_PHAvw_RC_2d = spec.E_ANGwu_RC.transpose();  E_PHAvw_RC_med = np.nanmedian(spec.E_PHAvw_RC, axis=0)
    E_PHAwu_RC_2d = spec.E_ANGwu_RC.transpose();  E_PHAwu_RC_med = np.nanmedian(spec.E_PHAwu_RC, axis=0)

    spec.EE_LC    = spec.EuEu_LC + spec.EvEv_LC + spec.EwEw_LC
    EuEu_LC_2d    = spec.EuEu_LC.transpose();     EuEu_NC_med = np.nanmedian(spec.EuEu_LC,    axis=0)
    EvEv_LC_2d    = spec.EvEv_LC.transpose();     EvEv_NC_med = np.nanmedian(spec.EvEv_LC,    axis=0)
    EwEw_LC_2d    = spec.EwEw_LC.transpose();     EwEw_NC_med = np.nanmedian(spec.EwEw_LC,    axis=0)
    EE_LC_2d      = spec.EE_LC.transpose();       EE_NC_med   = np.nanmedian(spec.EE_LC,      axis=0)
    E_DoPuv_LC_2d = spec.E_DoLuv_LC.transpose();  E_DoPuv_NC_med = np.nanmedian(spec.E_DoPuv_LC, axis=0)
    E_DoPvw_LC_2d = spec.E_DoLvw_LC.transpose();  E_DoPvw_NC_med = np.nanmedian(spec.E_DoPvw_LC, axis=0)
    E_DoPwu_LC_2d = spec.E_DoLwu_LC.transpose();  E_DoPwu_NC_med = np.nanmedian(spec.E_DoPwu_LC, axis=0)
    E_DoLuv_LC_2d = spec.E_DoLuv_LC.transpose();  E_DoLuv_NC_med = np.nanmedian(spec.E_DoLuv_LC, axis=0)
    E_DoLvw_LC_2d = spec.E_DoLvw_LC.transpose();  E_DoLvw_NC_med = np.nanmedian(spec.E_DoLvw_LC, axis=0)
    E_DoLwu_LC_2d = spec.E_DoLwu_LC.transpose();  E_DoLwu_NC_med = np.nanmedian(spec.E_DoLwu_LC, axis=0)
    E_DoCuv_LC_2d = spec.E_DoCuv_LC.transpose();  E_DoCuv_NC_med = np.nanmedian(spec.E_DoCuv_LC, axis=0)
    E_DoCvw_LC_2d = spec.E_DoCvw_LC.transpose();  E_DoCvw_NC_med = np.nanmedian(spec.E_DoCvw_LC, axis=0)
    E_DoCwu_LC_2d = spec.E_DoCwu_LC.transpose();  E_DoCwu_NC_med = np.nanmedian(spec.E_DoCwu_LC, axis=0)
    E_ANGuv_LC_2d = spec.E_ANGuv_LC.transpose();  E_ANGuv_NC_med = np.nanmedian(spec.E_ANGuv_LC, axis=0)
    E_ANGvw_LC_2d = spec.E_ANGvw_LC.transpose();  E_ANGvw_NC_med = np.nanmedian(spec.E_ANGvw_LC, axis=0)
    E_ANGwu_LC_2d = spec.E_ANGwu_LC.transpose();  E_ANGwu_NC_med = np.nanmedian(spec.E_ANGwu_LC, axis=0)
    E_COHuv_LC_2d = spec.E_ANGwu_LC.transpose();  E_COHuv_NC_med = np.nanmedian(spec.E_COHuv_LC, axis=0)
    E_COHvw_LC_2d = spec.E_ANGwu_LC.transpose();  E_COHvw_NC_med = np.nanmedian(spec.E_COHvw_LC, axis=0)
    E_COHwu_LC_2d = spec.E_ANGwu_LC.transpose();  E_COHwu_NC_med = np.nanmedian(spec.E_COHwu_LC, axis=0)
    E_PHAuv_LC_2d = spec.E_ANGwu_LC.transpose();  E_PHAuv_NC_med = np.nanmedian(spec.E_PHAuv_LC, axis=0)
    E_PHAvw_LC_2d = spec.E_ANGwu_LC.transpose();  E_PHAvw_NC_med = np.nanmedian(spec.E_PHAvw_LC, axis=0)
    E_PHAwu_LC_2d = spec.E_ANGwu_LC.transpose();  E_PHAwu_NC_med = np.nanmedian(spec.E_PHAwu_LC, axis=0)

f_min0 = freq_1d[0];  f_max0 = freq_1d[N_frequency-1]
print(spec.str_unit, "   f_MAX/min:", f_max0, f_min0, "   num of f, step, window:", N_frequency, data.freq_step[0][0], data.freq_width[0][0])

## Spec

In [ ]:
n0 = 0;  n1 = n_time0//2;   n2 = n_time0-1
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(14, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)
else:
    ax1 = fig.add_subplot(1, 1, 1)

ax1.plot(freq_1d, spec.EuEu[n0], ':r',  linewidth=1.0, label='uu init')
ax1.plot(freq_1d, spec.EvEv[n0], ':g',  linewidth=0.8, label='vv init')
ax1.plot(freq_1d, spec.EwEw[n0], ':b',  linewidth=0.6, label='ww init')
ax1.plot(freq_1d, spec.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq_1d, spec.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq_1d, spec.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq_1d, spec.EuEu[n2], '-r',  linewidth=1.0, label='uu end')
ax1.plot(freq_1d, spec.EvEv[n2], '-g',  linewidth=0.8, label='vv end')
ax1.plot(freq_1d, spec.EwEw[n2], '-b',  linewidth=0.6, label='ww end')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')
#
if comp_mode > 0:    # Matrix
    ax2.plot(freq_1d, spec.E_DoLuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax2.plot(freq_1d, spec.E_DoLvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax2.plot(freq_1d, spec.E_DoLwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax2.plot(freq_1d, spec.E_DoLuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax2.plot(freq_1d, spec.E_DoLvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax2.plot(freq_1d, spec.E_DoLwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax2.plot(freq_1d, spec.E_DoLuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax2.plot(freq_1d, spec.E_DoLvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax2.plot(freq_1d, spec.E_DoLwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax3.plot(freq_1d, spec.E_DoCuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax3.plot(freq_1d, spec.E_DoCvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax3.plot(freq_1d, spec.E_DoCwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax3.plot(freq_1d, spec.E_DoCuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax3.plot(freq_1d, spec.E_DoCvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax3.plot(freq_1d, spec.E_DoCwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax3.plot(freq_1d, spec.E_DoCuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax3.plot(freq_1d, spec.E_DoCvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax3.plot(freq_1d, spec.E_DoCwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax4.plot(freq_1d, spec.E_ANGuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax4.plot(freq_1d, spec.E_ANGvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax4.plot(freq_1d, spec.E_ANGwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax4.plot(freq_1d, spec.E_ANGuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax4.plot(freq_1d, spec.E_ANGvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax4.plot(freq_1d, spec.E_ANGwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax4.plot(freq_1d, spec.E_ANGuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax4.plot(freq_1d, spec.E_ANGvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax4.plot(freq_1d, spec.E_ANGwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax5.plot(freq_1d, spec.E_DoLuv[n0], ':r',  linewidth=1.0, label='uv init')
    if f_mode == 1:
        ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

ax1.set_ylabel(spec.str_unit)
if comp_mode > 0:    # Matrix
    ax4.set_xlabel('frequency [Hz]');  ax2.set_ylabel('Deg. Linear Pol.');  ax3.set_ylabel('Deg. Circular Pol.');  ax4.set_ylabel('Linear Deg.')    
else:
    ax1.set_xlabel('frequency [Hz]')

date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)
if comp_mode > 0:
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); ax4.legend(loc='upper right', fontsize=8) 

xlim=[f_min, f_max];  ax1.set_xlim(xlim)
if comp_mode > 0:    # Matrix
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim)
ylim=[10**p_min, 10**p_max]; ax1.set_ylim(ylim)
if comp_mode > 0:    # Matrix
    ylim=[-0.05, 1.05]; ax2.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax3.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax4.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

In [ ]:
p_min = p_min0;  p_max = p_max0
n0 = 0;  n1 = n_time0//4;   n2 = n_time0-1
# n0 = 64;  n1 = 65;   n2 = 66
f_min = f_min0;  f_max = f_max0
# f_min = 80;       f_max = 2068

fig = plt.figure(figsize=(14, 11));  ax1 = fig.add_subplot(1, 1, 1)
#ax1.plot(freq_1d, EuEu_med,      '-r',  linewidth=0.5, label='EuEu_med')
#ax1.plot(freq_1d, EvEv_med,      '-g',  linewidth=0.5, label='EvEv_med')
#ax1.plot(freq_1d, EwEw_med,      '-b',  linewidth=0.5, label='EwEw_med')
ax1.plot(freq_1d, spec.EuEu[n0], '-.r',  linewidth=0.5, label='uu init')
ax1.plot(freq_1d, spec.EvEv[n0], '-.g',  linewidth=0.5, label='vv init')
ax1.plot(freq_1d, spec.EwEw[n0], '-.b',  linewidth=0.5, label='ww init')
ax1.plot(freq_1d, spec.EuEu[n1], '-r', linewidth=2.0, label='uu half')
ax1.plot(freq_1d, spec.EvEv[n1], '-g', linewidth=2.0, label='vv half')
ax1.plot(freq_1d, spec.EwEw[n1], '-b', linewidth=2.0, label='ww half')
ax1.plot(freq_1d, spec.EuEu[n2], ':r',  linewidth=2.0, label='uu end')
ax1.plot(freq_1d, spec.EvEv[n2], ':g',  linewidth=2.0, label='vv end')
ax1.plot(freq_1d, spec.EwEw[n2], ':b',  linewidth=2.0, label='ww end')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')

ax1.set_ylabel(spec.str_unit); ax1.set_xlabel('frequency [Hz]')
date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max]
ax1.set_xlim(xlim)
ylim=[10**p_min, 10**p_max]; 
ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec2.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec2-log.png'
    fig.savefig(png_fname)

## FT

In [ ]:
p_min = p_min0;  p_max = p_max0
# p_min = -14.5;     p_max = -13.5
f_min = f_min0;  f_max = f_max0
# f_min = 300;     f_max = 3000
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)
    ax4.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)
    ax3.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max);   ax2.set_ylim(f_min, f_max);   ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')
ax1.set_ylabel('EuEu [kHz]'); ax2.set_ylabel('EvEv [kHz]'); ax3.set_ylabel('EwEw [kHz]')
ax1.set_title("[" + data_name + "]")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str_unit)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(spec.str_unit)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    if t_min0 == 0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        xlim=[t_min0, t_max0]
    #E_min = '2024-08-21 04:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    #E_max = '2024-08-21 19:00:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode < 1:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep NC
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max
    num_1d = np.arange(n_time0)

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)
        ax4.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)
        ax3.set_xlabel(str_date)
    ax1.set_ylim(f_min, f_max);   ax2.set_ylim(f_min, f_max);   ax3.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')
    ax1.set_ylabel('EuEu NC [kHz]'); ax2.set_ylabel('EvEv NC [kHz]'); ax3.set_ylabel('EwEw NC [kHz]')
    ax1.set_title("[" + data_name + "]")

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
        pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)
    pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str_unit)
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(spec.str_unit)

    print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax4.set_xlim(xlim)
    else:
        if t_min0 == 0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_FT_NC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_FT_NC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep RC
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max
    num_1d = np.arange(n_time0)

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)
        ax4.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)
        ax3.set_xlabel(str_date)
    ax1.set_ylim(f_min, f_max);   ax2.set_ylim(f_min, f_max);   ax3.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')
    ax1.set_ylabel('EuEu RC [kHz]'); ax2.set_ylabel('EvEv RC [kHz]'); ax3.set_ylabel('EwEw RC [kHz]')
    ax1.set_title("[" + data_name + "]")

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
        pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)
    pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str_unit)
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(spec.str_unit)

    print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax4.set_xlim(xlim)
    else:
        if t_min0 == 0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_FT_RC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_FT_RC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep LC
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max
    num_1d = np.arange(n_time0)

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)
        ax4.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)
        ax3.set_xlabel(str_date)
    ax1.set_ylim(f_min, f_max);   ax2.set_ylim(f_min, f_max);   ax3.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')
    ax1.set_ylabel('EuEu LC [kHz]'); ax2.set_ylabel('EvEv LC [kHz]'); ax3.set_ylabel('EwEw LC [kHz]')
    ax1.set_title("[" + data_name + "]")

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
        pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)
    pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str_unit)
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(spec.str_unit)

    print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax4.set_xlim(xlim)
    else:
        if t_min0 == 0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_FT_LC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_FT_LC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
p_min = p_min0;  p_max = p_max0
#p_min = -14;     p_max = -13
f_min = f_min0;  f_max = f_max0
# f_min = 300;     f_max = 3000
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(2, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(1, 1, 1);                                     ax1.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log')
ax1.set_ylabel('EE [kHz]');  ax1.set_title("[" + data_name + "]  ")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp1 = fig2d.colorbar(p1, ax=ax2, orientation="vertical"); pp1.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]; ax2.set_xlim(xlim) 
else:
    if t_min0 == 0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        xlim=[t_min0, t_max0]
    # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT2.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT2-log.png'
    fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep NC
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(1, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(1, 1, 1);                                     ax1.set_xlabel(str_date)
    ax1.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log')
    ax1.set_ylabel('EE NC [kHz]');  ax1.set_title("[" + data_name + "]  ")

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, EE_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax4.plot(np.ravel(data.epoch[:]), '.')
        pp1 = fig2d.colorbar(p1, ax=ax2, orientation="vertical"); pp1.set_label(spec.str_unit)
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)

    print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]; ax2.set_xlim(xlim) 
    else:
        if t_min0 == 0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_FT2_NC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_FT2_NC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep RC
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(1, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(1, 1, 1);                                     ax1.set_xlabel(str_date)
    ax1.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log')
    ax1.set_ylabel('EE RC [kHz]');  ax1.set_title("[" + data_name + "]  ")

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, EE_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax4.plot(np.ravel(data.epoch[:]), '.')
        pp1 = fig2d.colorbar(p1, ax=ax2, orientation="vertical"); pp1.set_label(spec.str_unit)
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)

    print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]; ax2.set_xlim(xlim) 
    else:
        if t_min0 == 0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_FT2_RC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_FT2_RC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep LC
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(1, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(1, 1, 1);                                     ax1.set_xlabel(str_date)
    ax1.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log')
    ax1.set_ylabel('EE LC [kHz]');  ax1.set_title("[" + data_name + "]  ")

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, EE_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax4.plot(np.ravel(data.epoch[:]), '.')
        pp1 = fig2d.colorbar(p1, ax=ax2, orientation="vertical"); pp1.set_label(spec.str_unit)
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)

    print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]; ax2.set_xlim(xlim) 
    else:
        if t_min0 == 0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_FT2_LC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_FT2_LC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # Matrix - polsep
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max
    num_1d = np.arange(n_time0)

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
        ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
        ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9)
        ax10 = fig2d.add_subplot(10, 1, 10);                                                                     ax10.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
        ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
        ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9);  ax9.set_xlabel(str_date)

    ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
    ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');  ax2.set_yscale('log');      ax3.set_yscale('log');      ax4.set_yscale('log');      ax5.set_yscale('log')
        ax6.set_yscale('log');  ax7.set_yscale('log');      ax8.set_yscale('log');      ax9.set_yscale('log')
    ax1.set_ylabel('EuEu NC[kHz]'); ax2.set_ylabel('RC[kHz]'); ax3.set_ylabel('LC[kHz]') 
    ax4.set_ylabel('EvEv NC[kHz]'); ax5.set_ylabel('RC[kHz]'); ax6.set_ylabel('LC[kHz]') 
    ax7.set_ylabel('EwEw NC[kHz]'); ax8.set_ylabel('RC[kHz]'); ax9.set_ylabel('LC[kHz]'); ax1.set_title("[" + data_name + "]"'')

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, EuEu_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, EuEu_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p4 = ax4.pcolormesh(num_1d, freq_1d, EvEv_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p5 = ax5.pcolormesh(num_1d, freq_1d, EvEv_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p6 = ax6.pcolormesh(num_1d, freq_1d, EvEv_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p7 = ax7.pcolormesh(num_1d, freq_1d, EwEw_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p8 = ax8.pcolormesh(num_1d, freq_1d, EwEw_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p9 = ax9.pcolormesh(num_1d, freq_1d, EwEw_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.')
        pp3 = fig2d.colorbar(p3, ax=ax10, orientation="vertical"); pp3.set_label(spec.str_unit)
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EuEu_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EuEu_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p4 = ax4.pcolormesh(Epoch_1d, freq_1d, EvEv_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p5 = ax5.pcolormesh(Epoch_1d, freq_1d, EvEv_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p6 = ax6.pcolormesh(Epoch_1d, freq_1d, EvEv_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p7 = ax7.pcolormesh(Epoch_1d, freq_1d, EwEw_NC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p8 = ax8.pcolormesh(Epoch_1d, freq_1d, EwEw_RC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
        p9 = ax9.pcolormesh(Epoch_1d, freq_1d, EwEw_LC_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")
    pp1.set_label(spec.str_unit); pp2.set_label(spec.str_unit); pp3.set_label(spec.str_unit)
    pp4.set_label(spec.str_unit); pp5.set_label(spec.str_unit); pp6.set_label(spec.str_unit)
    pp7.set_label(spec.str_unit); pp8.set_label(spec.str_unit); pp9.set_label(spec.str_unit)

    print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax10.set_xlim(xlim)
    else:
        if t_min0==0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_FT_polsep.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_FT_polsep-log.png'
        fig2d.savefig(png_fname)

### specific duration: median & peak (special)

In [ ]:
"""
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0
# f_min = 80;       f_max = 2068

n_num1 = 0;    n_num2 = -1

print(Epoch_1d[820], Epoch_1d[900])
EuEu_med2    = np.nanmedian(spec.EuEu[820:900],    axis=0)
EvEv_med2    = np.nanmedian(spec.EvEv[820:900],    axis=0)
EwEw_med2    = np.nanmedian(spec.EwEw[820:900],    axis=0)
EE_med2      = np.nanmedian(spec.EuEu[820:900],    axis=0)
EuEu_max2    = np.nanmax   (spec.EuEu[820:900],    axis=0)
EvEv_max2    = np.nanmax   (spec.EvEv[820:900],    axis=0)
EwEw_max2    = np.nanmax   (spec.EwEw[820:900],    axis=0)
EE_max2      = np.nanmax   (spec.EuEu[820:900],    axis=0)

fig = plt.figure(figsize=(14, 11));  ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(freq_1d, EE_med2  ,      '-k',  linewidth=4.0, label='EE_med')
ax1.plot(freq_1d, EuEu_med2,      '-r',  linewidth=2.0, label='EuEu_med')
ax1.plot(freq_1d, EvEv_med2,      '-g',  linewidth=2.0, label='EvEv_med')
ax1.plot(freq_1d, EwEw_med2,      '-b',  linewidth=2.0, label='EwEw_med')
ax1.plot(freq_1d, EE_max2  ,      ':k',  linewidth=1.0, label='EE_max')
ax1.plot(freq_1d, EuEu_max2,      ':r',  linewidth=1.0, label='EuEu_max')
ax1.plot(freq_1d, EvEv_max2,      ':g',  linewidth=1.0, label='EvEv_max')
ax1.plot(freq_1d, EwEw_max2,      ':b',  linewidth=1.0, label='EwEw_max')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')

ax1.set_ylabel(spec.str_unit); ax1.set_xlabel('frequency [Hz]')
date1 = data.epoch[n_num1];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n_num2];  date2 = date2.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2;  ax1.set_title(title_date)
ax1.legend(loc='upper left', fontsize=8)

xlim=[f_min, f_max];           ax1.set_xlim(xlim)
#ylim=[10**p_min, 10**p_max];  ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec2.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec2-log.png'
    fig.savefig(png_fname)
"""

## Ang-T

In [ ]:
if comp_mode > 0:    # Matrix
    p_min = p_min0;      p_max = p_max0
    f_min = f_min0;      f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
        ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
        ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9)
        ax10 = fig2d.add_subplot(10, 1, 10);                                                                     ax10.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
        ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
        ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9);  ax9.set_xlabel(str_date)

    ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
    ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');  ax2.set_yscale('log');      ax3.set_yscale('log');      ax4.set_yscale('log');      ax5.set_yscale('log')
        ax6.set_yscale('log');  ax7.set_yscale('log');      ax8.set_yscale('log');      ax9.set_yscale('log')
    ax1.set_ylabel('D.Lin UV'); ax2.set_ylabel('VW'); ax3.set_ylabel('WU')
    ax4.set_ylabel('D.Cir UV'); ax5.set_ylabel('VW'); ax6.set_ylabel('WU')
    ax7.set_ylabel('D.Ang UV'); ax8.set_ylabel('VW'); ax9.set_ylabel('WU'); ax1.set_title("[" + data_name + "]"'')

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(num_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(num_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(num_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.');   pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical")
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(Epoch_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(Epoch_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(Epoch_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")
    
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax10.set_xlim(xlim)
    else:
        if t_min0==0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_AngT.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_AngT-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep NC
    p_min = p_min0;      p_max = p_max0
    f_min = f_min0;      f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
        ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
        ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9)
        ax10 = fig2d.add_subplot(10, 1, 10);                                                                     ax10.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
        ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
        ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9);  ax9.set_xlabel(str_date)

    ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
    ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');  ax2.set_yscale('log');      ax3.set_yscale('log');      ax4.set_yscale('log');      ax5.set_yscale('log')
        ax6.set_yscale('log');  ax7.set_yscale('log');      ax8.set_yscale('log');      ax9.set_yscale('log')
    ax1.set_ylabel('Lin UV NC'); ax2.set_ylabel('VW NC'); ax3.set_ylabel('WU NC')
    ax4.set_ylabel('Cir UV NC'); ax5.set_ylabel('VW NC'); ax6.set_ylabel('WU NC')
    ax7.set_ylabel('Ang UV NC'); ax8.set_ylabel('VW NC'); ax9.set_ylabel('WU NC'); ax1.set_title("[" + data_name + "]"'')

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoLuv_NC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoLvw_NC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoLwu_NC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCuv_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCwu_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(num_1d, freq_1d, E_ANGuv_NC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(num_1d, freq_1d, E_ANGvw_NC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(num_1d, freq_1d, E_ANGwu_NC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.');   pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical")
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLuv_NC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoLvw_NC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_DoLwu_NC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(Epoch_1d, freq_1d, E_ANGuv_NC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(Epoch_1d, freq_1d, E_ANGvw_NC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(Epoch_1d, freq_1d, E_ANGwu_NC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")
    
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax10.set_xlim(xlim)
    else:
        if t_min0==0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_AngT_NC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_AngT_NC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep RC
    p_min = p_min0;      p_max = p_max0
    f_min = f_min0;      f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
        ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
        ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9)
        ax10 = fig2d.add_subplot(10, 1, 10);                                                                     ax10.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
        ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
        ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9);  ax9.set_xlabel(str_date)

    ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
    ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');  ax2.set_yscale('log');      ax3.set_yscale('log');      ax4.set_yscale('log');      ax5.set_yscale('log')
        ax6.set_yscale('log');  ax7.set_yscale('log');      ax8.set_yscale('log');      ax9.set_yscale('log')
    ax1.set_ylabel('Lin UV RC'); ax2.set_ylabel('VW RC'); ax3.set_ylabel('WU RC')
    ax4.set_ylabel('Cir UV RC'); ax5.set_ylabel('VW RC'); ax6.set_ylabel('WU RC')
    ax7.set_ylabel('Ang UV RC'); ax8.set_ylabel('VW RC'); ax9.set_ylabel('WU RC'); ax1.set_title("[" + data_name + "]"'')

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoLuv_RC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoLvw_RC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoLwu_RC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCuv_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCwu_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(num_1d, freq_1d, E_ANGuv_RC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(num_1d, freq_1d, E_ANGvw_RC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(num_1d, freq_1d, E_ANGwu_RC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.');   pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical")
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLuv_RC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoLvw_RC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_DoLwu_RC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(Epoch_1d, freq_1d, E_ANGuv_RC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(Epoch_1d, freq_1d, E_ANGvw_RC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(Epoch_1d, freq_1d, E_ANGwu_RC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")
    
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax10.set_xlim(xlim)
    else:
        if t_min0==0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_AngT_RC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_AngT_RC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # PolSep LC
    p_min = p_min0;      p_max = p_max0
    f_min = f_min0;      f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
        ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
        ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9)
        ax10 = fig2d.add_subplot(10, 1, 10);                                                                     ax10.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
        ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
        ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9);  ax9.set_xlabel(str_date)

    ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
    ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');  ax2.set_yscale('log');      ax3.set_yscale('log');      ax4.set_yscale('log');      ax5.set_yscale('log')
        ax6.set_yscale('log');  ax7.set_yscale('log');      ax8.set_yscale('log');      ax9.set_yscale('log')
    ax1.set_ylabel('Lin UV LC'); ax2.set_ylabel('VW LC'); ax3.set_ylabel('WU LC')
    ax4.set_ylabel('Cir UV LC'); ax5.set_ylabel('VW LC'); ax6.set_ylabel('WU LC')
    ax7.set_ylabel('Ang UV LC'); ax8.set_ylabel('VW LC'); ax9.set_ylabel('WU LC'); ax1.set_title("[" + data_name + "]"'')

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoLuv_LC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoLvw_LC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoLwu_LC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCuv_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCwu_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(num_1d, freq_1d, E_ANGuv_LC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(num_1d, freq_1d, E_ANGvw_LC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(num_1d, freq_1d, E_ANGwu_LC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.');   pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical")
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLuv_LC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoLvw_LC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_DoLwu_LC_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(Epoch_1d, freq_1d, E_ANGuv_LC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(Epoch_1d, freq_1d, E_ANGvw_LC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(Epoch_1d, freq_1d, E_ANGwu_LC_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")
    
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax10.set_xlim(xlim)
    else:
        if t_min0==0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_AngT_LC.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_AngT_LC-log.png'
        fig2d.savefig(png_fname)

In [ ]:
if comp_mode == 2:    # Matrix - polsep
    p_min = p_min0;      p_max = p_max0
    f_min = f_min0;      f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
        ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
        ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9)
        ax10 = fig2d.add_subplot(10, 1, 10);                                                                     ax10.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
        ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
        ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9);  ax9.set_xlabel(str_date)

    ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
    ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');  ax2.set_yscale('log');      ax3.set_yscale('log');      ax4.set_yscale('log');      ax5.set_yscale('log')
        ax6.set_yscale('log');  ax7.set_yscale('log');      ax8.set_yscale('log');      ax9.set_yscale('log')
    ax1.set_ylabel('Cir UV NC'); ax2.set_ylabel('RC'); ax3.set_ylabel('LC')
    ax4.set_ylabel('Cir VW NC'); ax5.set_ylabel('RC'); ax6.set_ylabel('LC')
    ax7.set_ylabel('Cir WU NC'); ax8.set_ylabel('RC'); ax9.set_ylabel('LC'); ax1.set_title("[" + data_name + "]"'')

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoCuv_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoCuv_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoCuv_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCvw_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCvw_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(num_1d, freq_1d, E_DoCwu_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p8 = ax8.pcolormesh(num_1d, freq_1d, E_DoCwu_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p9 = ax9.pcolormesh(num_1d, freq_1d, E_DoCwu_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.');   pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical")
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p4 = ax4.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_NC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p8 = ax8.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_RC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p9 = ax9.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_LC_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")
    
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax10.set_xlim(xlim)
    else:
        if t_min0==0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            xlim=[t_min0, t_max0]
        # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_AngT_polsep.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_AngT_polsep-log.png'
        fig2d.savefig(png_fname)

In [ ]:
# FT & Cir
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0
f_min = 80;      f_max = 1000
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3);  ax3.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(2, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max);            ax2.set_ylim(f_min, f_max); 
if f_mode == 1:
    ax1.set_yscale('log');             ax2.set_yscale('log')
ax1.set_ylabel('EvEv [kHz]');          ax2.set_ylabel('D.Cir VW');        ax1.set_title("[" + data_name + "]  ")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d,   freq_1d, EE_2d,      norm=colors.LogNorm  (vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d,   freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01,     vmax=1.01),      cmap='bwr')
    p3 = ax3.plot(np.ravel(data.epoch[:]), '.')
    pp1 = fig2d.colorbar(p1, ax=ax3, orientation="vertical"); pp1.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_2d,    norm=colors.LogNorm  (vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01,     vmax=1.01),      cmap='bwr')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");     pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")

# print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]; ax3.set_xlim(xlim) 
else:
    if t_min0 == 0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        xlim=[t_min0, t_max0]
    # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-Cir.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-Cir-log.png'
    fig2d.savefig(png_fname)

# CLEANED data

In [ ]:
# RFI rejection:  Mode -- 0:median   1:max reject   2:min
EuEu_c = juice_math.clean_rfi(np.ravel(spec.EuEu), 5, 2);  EvEv_c = juice_math.clean_rfi(np.ravel(spec.EvEv), 5, 2)
EwEw_c = juice_math.clean_rfi(np.ravel(spec.EwEw), 5, 2);  EE_c   = juice_math.clean_rfi(np.ravel(spec.EE),   5, 2)
EuEu_c = EuEu_c.reshape(n_time0, n_freq1);                 EvEv_c = EvEv_c.reshape(n_time0, n_freq1)
EwEw_c = EwEw_c.reshape(n_time0, n_freq1);                 EE_c   = EE_c.reshape  (n_time0, n_freq1)
EuEu_c_2d = EuEu_c.transpose();  EvEv_c_2d = EvEv_c.transpose();  EwEw_c_2d = EwEw_c.transpose();  EE_c_2d = EE_c.transpose()

## CLEANED - Spec

In [ ]:
n0 = 0;  n1 = n_time0//2;  n2 = n_time0-1
p_min = p_min0;  p_max = p_max0
p_min = -15;     p_max = -10
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(freq_1d, spec.EuEu[n0], ':r',  linewidth=1.0, label='uu init')
ax1.plot(freq_1d, spec.EvEv[n0], ':g',  linewidth=0.8, label='vv init')
ax1.plot(freq_1d, spec.EwEw[n0], ':b',  linewidth=0.6, label='ww init')
ax1.plot(freq_1d, spec.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq_1d, spec.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq_1d, spec.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq_1d, spec.EuEu[n2], '-r',  linewidth=1.0, label='uu end')
ax1.plot(freq_1d, spec.EvEv[n2], '-g',  linewidth=0.8, label='vv end')
ax1.plot(freq_1d, spec.EwEw[n2], '-b',  linewidth=0.6, label='ww end')
ax2.plot(freq_1d, EuEu_c[n0],    ':r',  linewidth=1.0, label='uu init cleaned')
ax2.plot(freq_1d, EvEv_c[n0],    ':g',  linewidth=0.8, label='vv init cleaned')
ax2.plot(freq_1d, EwEw_c[n0],    ':b',  linewidth=0.6, label='ww init cleaned')
ax2.plot(freq_1d, EuEu_c[n1],    '-.r', linewidth=1.0, label='uu half cleaned')
ax2.plot(freq_1d, EvEv_c[n1],    '-.g', linewidth=0.8, label='vv half cleaned')
ax2.plot(freq_1d, EwEw_c[n1],    '-.b', linewidth=0.6, label='ww half cleaned')
ax2.plot(freq_1d, EuEu_c[n2],    '-r',  linewidth=1.0, label='uu end cleaned')
ax2.plot(freq_1d, EvEv_c[n2],    '-g',  linewidth=0.8, label='vv end cleaned')
ax2.plot(freq_1d, EwEw_c[n2],    '-b',  linewidth=0.6, label='ww end cleaned')
ax3.plot(freq_1d, spec.EE[n0],   ':r',  linewidth=1.0, label='EE init')
ax3.plot(freq_1d, spec.EE[n1],   ':g',  linewidth=1.0, label='EE half')
ax3.plot(freq_1d, spec.EE[n2],   ':b',  linewidth=1.0, label='EE end')
ax3.plot(freq_1d, EE_c[n0],      '-r',  linewidth=1.0, label='EE init cleaned')
ax3.plot(freq_1d, EE_c[n1],      '-g',  linewidth=1.0, label='EE half cleaned')
ax3.plot(freq_1d, EE_c[n2],      '-b',  linewidth=1.0, label='EE end cleaned')
ax1.set_yscale('log');      ax2.set_yscale('log');     ax3.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');     ax3.set_xscale('log')
ax1.set_ylabel(spec.str_unit);   ax2.set_ylabel(spec.str_unit);  ax3.set_xlabel('frequency [Hz]')

date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title("[" + data_name + "]  " + title_date)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max];          ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)
# ylim=[10**p_min, 10**p_max];  ax1.set_ylim(ylim); ax2.set_ylim(ylim); ax3.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log-c.png'
        # fig.savefig(png_fname

In [ ]:
n0 = 0;  n1 = n_time0//4;  n2 = n_time0-1
f_min = f_min0;  f_max = f_max0
p_min = p_min0;  p_max = p_max0

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(freq_1d, spec.EvEv[n0]/spec.EuEu[n0], ':r',  linewidth=0.8, label='V/U init')
ax1.plot(freq_1d, spec.EwEw[n0]/spec.EuEu[n0], ':b',  linewidth=0.6, label='W/U init')
ax1.plot(freq_1d, spec.EvEv[n1]/spec.EuEu[n1], '-.r', linewidth=0.8, label='V/U half')
ax1.plot(freq_1d, spec.EwEw[n1]/spec.EuEu[n1], '-.b', linewidth=0.6, label='W/U half')
ax1.plot(freq_1d, spec.EvEv[n2]/spec.EuEu[n2], '-r',  linewidth=0.8, label='V/U end')
ax1.plot(freq_1d, spec.EwEw[n2]/spec.EuEu[n2], '-b',  linewidth=0.6, label='W/U end')
ax2.plot(freq_1d, EvEv_c[n0]/EuEu_c[n0],       ':r',  linewidth=0.8, label='V/U init cleaned')
ax2.plot(freq_1d, EwEw_c[n0]/EuEu_c[n0],       ':b',  linewidth=0.6, label='W/U init cleaned')
ax2.plot(freq_1d, EvEv_c[n1]/EuEu_c[n1],       '-.r', linewidth=0.8, label='V/U half cleaned')
ax2.plot(freq_1d, EwEw_c[n1]/EuEu_c[n1],       '-.b', linewidth=0.6, label='W/U half cleaned')
ax2.plot(freq_1d, EvEv_c[n2]/EuEu_c[n2],       '-r',  linewidth=0.8, label='V/U end cleaned')
ax2.plot(freq_1d, EwEw_c[n2]/EuEu_c[n2],       '-b',  linewidth=0.6, label='W/U end cleaned')
ax3.plot(freq_1d, spec.EE[n0],   ':r',  linewidth=1.0, label='EE init')
ax3.plot(freq_1d, spec.EE[n1],   ':g',  linewidth=1.0, label='EE half')
ax3.plot(freq_1d, spec.EE[n2],   ':b',  linewidth=1.0, label='EE end')
ax3.plot(freq_1d, EE_c[n0],      '-r',  linewidth=1.0, label='EE init cleaned')
ax3.plot(freq_1d, EE_c[n1],      '-g',  linewidth=1.0, label='EE half cleaned')
ax3.plot(freq_1d, EE_c[n2],      '-b',  linewidth=1.0, label='EE end cleaned')
ax1.set_yscale('log');      ax2.set_yscale('log');     ax3.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');     ax3.set_xscale('log')
ax1.set_ylabel("ratio");    ax2.set_ylabel("ratio - cleaned");   ax3.set_ylabel(spec.str_unit);  ax3.set_xlabel('frequency [Hz]')

date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title("[" + data_name + "]  " + title_date)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max];          ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)
# ylim=[10**p_min, 10**p_max];  ax1.set_ylim(ylim); ax2.set_ylim(ylim); ax3.set_ylim(ylim)

fig.subplots_adjust(hspace=0); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_ratio.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_ratio-log.png'
    # fig2d.savefig(png_fname)

## CLEANED -- FT

In [ ]:
p_min = p_min0+1.5;  p_max = p_max0
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)
    ax4.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)
    ax3.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max);   ax2.set_ylim(f_min, f_max);   ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')
ax1.set_ylabel('EuEu cleaned[kHz]'); ax2.set_ylabel('EvEv cleaned [kHz]'); ax3.set_ylabel('EwEw cleaned [kHz]')
ax1.set_title("[" + data_name + "]")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str_unit)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(spec.str_unit)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    if t_min0==0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        xlim=[t_min0, t_max0]
    # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-c-log.png'
    fig2d.savefig(png_fname)

In [ ]:
p_min = p_min0;  p_max = p_max0
# p_min = p_min0+1.5;  p_max = p_max0-1.5
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
f_min = 800;  f_max = f_max0

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(1, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(1, 1, 1);                                     ax1.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log')
ax1.set_ylabel('EE [kHz]');  ax1.set_title("[" + data_name + "]  ")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp1 = fig2d.colorbar(p1, ax=ax2, orientation="vertical"); pp1.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    if t_min0==0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        xlim=[t_min0, t_max0]
    # E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    # E_max = '2024-08-21 03:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT2-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT2-c-log.png'
    fig2d.savefig(png_fname)

In [ ]:
"""
import csv
with open("sid3-f.csv", 'w') as f:
    writer = csv.writer(f)
    for i in range(freq_1d.shape[0]):
        writer.writerow([ i, data.frequency[0][i], data.freq_width[0][i], data.freq_step[0][i]])
print(freq_1d.shape[0], data.frequency.shape)
"""